# **[PyTorch Geometric] PyG Data Format of Halide Perovskites -> Custom GNN Modeling -> Training**

1. **Material Project Dataset (energy per atom) --> PyTorch Geometric Data Format 변환**
> Data Format의 Undirected 여부가 일정하지 않는 문제 존재 --> torch.closeall()의 문제. 모델학습에는 영향을 주지 않음! <br>
> 아마도 torch.close()에서 소수점 이하를 잘 구별하지 못하는 듯 <br>
> round 등의 함수로 소수점 일정수준으로 정해야 한다. <br>

2. **PyG MessagePassing Process [Message --> Aggregation --> Update]로 Custom Layer / Custom GNN Modeling**
> Best Validation Performance Model 저장하는 방법 찾기 <br>

3. **Benchmark Study for Each GNN Models**
> Custom **GAT** (Static Attention) <br>
> Custom **GATv2** (Dynamic Attention) <br>
> Custom **GIN** (Graph Isomorphism Network) <br>
> Custom **GraphConv** (Weisfeilr-Lepman Neural Network) <br>
> Hierarchical Graph convolution network (real custom for my own) <br>



## Installation Required Packages for PyTorch Geometric / Pymatgen



In [1]:
# Install Required Packages
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install -q pymatgen==2020.11.11 

     |████████████████████████████████| 7.9 MB 776 kB/s 
     |████████████████████████████████| 3.5 MB 31.9 MB/s 
     |████████████████████████████████| 2.8 MB 19.4 MB/s 
     |████████████████████████████████| 109 kB 47.2 MB/s 
     |████████████████████████████████| 65 kB 5.0 MB/s 
     |████████████████████████████████| 38.1 MB 1.9 MB/s 
     |████████████████████████████████| 292 kB 58.6 MB/s 
     |████████████████████████████████| 98 kB 7.3 MB/s 
     |████████████████████████████████| 546 kB 45.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


## Google Drive Mount



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/
%ls -l

## Load Halide Perovskite Dataset

* Nurion 서버에서 생성한 7200개의 Halide Perovskite - Free Energy per atom 데이터셋을 사용하여 진행
> 약 55000개의 Nonlocal Data point로 보고 학습 <br>
> 이후 Sequence 데이터로 보고 학습시도할 수도 있다. <br>

* 일단 자체생성한 Nonlocal Halide Perovskite 데이터셋에 한정해서 물성예측이 가능한지를 먼저 시도

* 안 될 경우, CGCNN 연구의 사례처럼 Local state perovskites 정보까지 먼저 representation model로 학습 후, fine tuning하는 방식으로 nonlocal state를 학습하도록 시도해볼 수 있다.

* 그것도 안 될 경우, Sequnece 형식으로 학습시도

* 아예 안된다면, MP 데이터에 한정해서 순수하게 CGCNN을 능가하는 모델링 Benchmark study 연구를 시도할 수도 있다.



In [ ]:
%cd '/content/drive/MyDrive/00. Github/17. [2022-1학기] Benchmark-Study-For-Halide-Perovskite-with-Graph-Neural-Networks'

## ============ Custom Dataset Generation ============

Regression

| File                        | Property         | Units    | Data Ref.                                                    | Model Ref.                                                   |
| --------------------------- | ---------------- | -------- | ------------------------------------------------------------ | ------------------------------------------------------------ |
| `formation-energy-per-atom` | Formation Energy | eV/atom  | [Jain et al.](https://aip.scitation.org/doi/10.1063/1.4812323) | [Xie et al.](https://link.aps.org/doi/10.1103/PhysRevLett.120.145301) |
| ` final-energy-per-atom`    | Absolute Energy  | eV/atom  | [Jain et al.](https://aip.scitation.org/doi/10.1063/1.4812323) | [Xie et al.](https://link.aps.org/doi/10.1103/PhysRevLett.120.145301) |
| `band-gap`                  | Band Gap         | eV       | [Jain et al.](https://aip.scitation.org/doi/10.1063/1.4812323) | [Xie et al.](https://link.aps.org/doi/10.1103/PhysRevLett.120.145301) |
| `efermi`                    | Fermi Energy     | eV/atom  | [Jain et al.](https://aip.scitation.org/doi/10.1063/1.4812323) | [Xie et al.](https://link.aps.org/doi/10.1103/PhysRevLett.120.145301) |
| `bulk-moduli`               | Bulk Moduli      | log(GPa) | [Jain et al.](https://aip.scitation.org/doi/10.1063/1.4812323) | [Xie et al.](https://link.aps.org/doi/10.1103/PhysRevLett.120.145301) |
| `shear-moduli`              | Shear Moduli     | log(GPa) | [Jain et al.](https://aip.scitation.org/doi/10.1063/1.4812323) | [Xie et al.](https://link.aps.org/doi/10.1103/PhysRevLett.120.145301) |
| `poisson-ratio`             | Poisson Ratio    | —        | [Jain et al.](https://aip.scitation.org/doi/10.1063/1.4812323) | [Xie et al.](https://link.aps.org/doi/10.1103/PhysRevLett.120.145301) |


Classification

| File                        | Positive | Negative      | Data Ref.                                                    | Model Ref.                                                   |
| --------------------------- | -------- | ------------- | ------------------------------------------------------------ | ------------------------------------------------------------ |
| `semi-metal-classification` | Metal    | Semiconductor | [Jain et al.](https://aip.scitation.org/doi/10.1063/1.4812323) | [Xie et al.](https://link.aps.org/doi/10.1103/PhysRevLett.120.145301) |


- CGCNN models (and machine learning models in general) can only generalize to crystals from the same ditribution as training data. It is adviced to check **Data Ref.** before using pre-trained models. For instance, Materials Project uses [ICSD](https://icsd.fiz-karlsruhe.de/search/index.xhtml;jsessionid=E3291AF7E25ED34B31B9AD5A9CBF80A1) structures as input, which includes experimentally synthesized crystal structures. Significant errors can be expected if a CGCNN model trained on Materials Project is used to predict the properties of imaginary, thermadynamically unstable crystals.
- CGCNN models have prediction errors.  It is adviced to check **Model Ref.** to understand their accuracy before using pre-trained models.


### Halide Perovskite Datset --> Graph-Structured Dataset

1. **Index 지정하여 임의의 유효한 CIF ID 선택한다**.
2. **CIF ID를 통해 임의의 sample 데이터에 접근가능**
3. **Sample data의 Structure / Target Property 정보에 접근한다**.
4. **Structure 정보에서 --> Node Attributes / Edge Index / Edge Attributes 3가지의 Graph 관련 데이터 추출**
5. **CIF ID와 Target Property 정보를 정리**
6. **단일 Sample에 대해 1~5번 과정을 할 수 있다면, 전체 valid CIF ID list에 대해서도 동일한 과정을 반복할 수 있고, 이를 통해 CustomDataset 클래스를 작성가능하다**.
7. **CustomDataset 클래스는 getitem 메서드에 의해 인덱스로 개별 데이터 접근이 가능해야 한다. --> PyTorch_Geometric에서는 약간 다를 수 있다**.
8. **CustomDataset 클래스를 torch_geometric.dataset.DataLoader에 넣어 Graph batch를 가능하게 한다**.



In [ ]:
# Load Data : 여기서 새로운 Target Property로 전처리된 데이터셋을 사용한다!!
# 2개의 pickle 확장자 파일을 load하여 합쳐야 한다!!
import pickle
with open('Nonlocal-Halide-Perovskites-Leftover-Free-Energy-Noiselevel-0.01.pickle', 'rb') as fr1:
    id_prop_data = pickle.load(fr1)
with open('Nonlocal-Halide-Perovskites-Leftover-Free-Energy-Noiselevel-0.05.pickle', 'rb') as fr2:
    id_prop_data2 = pickle.load(fr2)
with open('Nonlocal-HalidePerovskite-MD-LeftoverFreeEnergy.pickle', 'rb') as fr3:
    id_prop_data3 = pickle.load(fr3)
       
print('Number of Halide Perovskite Crystal data 0.01 : {}'.format(len(id_prop_data)))
print('Number of Halide Perovskite Crystal data 0.05: {}'.format(len(id_prop_data2)))
print('Number of Halide Perovskite Crystal MD data : {}'.format(len(id_prop_data3)))

In [ ]:
# 2개의 data pointset을 합쳐서 하나로 만든다. extend로 이어붙인다!
id_prop_data.extend(id_prop_data2)
id_prop_data.extend(id_prop_data3)

del id_prop_data2 # 메모리 제거
del id_prop_data3
print(len(id_prop_data))

In [ ]:
# Check
idx = 7
crystal, target = id_prop_data[idx]
print('Target : {}'.format(target))
crystal 

In [ ]:
new_id_prop_data = []
for idx in range(len(id_prop_data)):
    crystal, target = id_prop_data[idx]

    if 1.6 <= abs(target) <= 3.6: # Feasible Target value에 대해서만 유효 데이터로 사용한다!
        temp = [crystal, target]
        new_id_prop_data.append(temp)

del id_prop_data # 변수제거하여 메모리 활용
print('Number of Normalized Dataset : {}'.format(len(new_id_prop_data)))

In [ ]:
idx = 7
crystal, target = new_id_prop_data[idx]
print(target)
crystal

In [ ]:
import random
for i in range(5):    
    random.shuffle(new_id_prop_data) # Data Shuffling 5 times
print('Shuffling Completed!!')

#### Single Sample Preprocessing Start



In [ ]:
idx = 7
crystal, target = new_id_prop_data[idx] # idx에 의해 Structure / Target Property 추출

print('Target Property : {}'.format(target))
print()
print('Structure Information :')
crystal

#### Initial Atom Embedding Vectors 

* Crystal Structure 구성원자들의 초기 임베딩 벡터 준비



In [ ]:
import json
with open('/content/drive/MyDrive/cgcnn/data/sample-regression/atom_init.json') as f:
    element_embedding = json.load(f)
print('Element Embedding Keys : {}'.format(element_embedding.keys())) # Atomic Numbers from 1 to 100

# All Embedding Vectors are 92-dimensional.
dimensions = []
for key, value in element_embedding.items():
    dimensions.append(len(value))
print('Dimension of Each Embedding Vectors : {}'.format(dimensions))

In [ ]:
# Key(Atomic Number) convert from 'string' to 'integer'.
element_embedding = {int(key) : value for key, value in element_embedding.items()}
print('Element Embedding Keys : {}'.format(element_embedding.keys()))

atom_types = set(element_embedding.keys())   # Set으로 중복원소 제거
print('Atom Types : {}'.format(atom_types))  # Set of Atomic Numbers from 1 to 100

In [ ]:
import numpy as np
# Initial Embedding Vectors per atoms --> Dictionary Saved.
embedding_dict = {}
for key, value in element_embedding.items():
    embedding_dict[key] = np.array(value, dtype = float) # value(list) --> np.array 변환

#### Node Attributes Matrix

* Crystal Structure 안의 모든 원자에 대한 Node Attributes Matrix 생성



In [ ]:
# Atomic Node Attributes
atomic_attributes = [embedding_dict[crystal[i].specie.number] for i in range(len(crystal))] # 결정의 i번째 원자번호로 임베딩벡터 Mapping
atomic_attributes = np.array(atomic_attributes)                                             # Atomic Node Attributes Matrix per Crystal

print('Atomic Attributes Shape : {}'.format(atomic_attributes.shape)) # Shape : [num_atoms, Initial Embedding Dimension (92)]
print(atomic_attributes)

1. **Unitcell 내부의 Central Node로부터 Cut-off Radius를 설정하고, 탐색영역 안에서 거리(Distance)에 따라 가까운 것부터 차례대로 정렬한다**.
2. **여기서 Radius는 충분히 크게 설정해도 상관없을 것 같다. 탐색범위가 넓어져도 그 안에서 거리기준으로 정렬한 후 M개 만큼만 이웃노드로 고려하기 때문이다. 오히려 Radius를 지나치게 작게 설정할 경우 (ex. radius = 4) 물질에 따라 이웃노드가 전혀 잡히지 않는 경우가 있었으므로 충분히 크게 잡아도 될 것 같다**.




In [ ]:
radius = 8    # Cut-off Radius [Angstrom]
all_neighbors = crystal.get_all_neighbors(r = radius, include_index = True )
# all_neighbors # a list of list of neighbors for each site in structure

In [ ]:
# x[1] : 유닛셀 안의 중심노드와 이웃노드 사이의 거리를 기반으로 정렬
all_neighbors_sorted = [sorted(neighbor, key = lambda x : x[1]) for neighbor in all_neighbors] # x[1] : 중심노드와 이웃노드 사이의 거리
# all_neighbors_sorted

In [ ]:
# 이웃노드들의 인덱스 행렬과 특성정보
neighbor_feature_index = []
neighbor_features = []
neighbor_coordinates = []
max_number_of_neighbors = 12  # M 

for neighbor in all_neighbors_sorted:

    neighbor_feature_index.append(list(map(lambda x : x[2], neighbor[:max_number_of_neighbors]))) # x[2] : 이웃원자 인덱스
    neighbor_features.append(list(map(lambda x : x[1], neighbor[:max_number_of_neighbors])))      # x[1] : 중심원자와 이웃원자 사이 거리
    neighbor_coordinates.append(list(map(lambda x : x.coords, neighbor[:max_number_of_neighbors]))) # x.coords : 이웃원자 좌표

# Convert to numpy arrays
neighbor_feature_index = np.array(neighbor_feature_index)
neighbor_features = np.array(neighbor_features)
neighbor_coordinates = np.array(neighbor_coordinates)

print('Neighbor Feature Index Shape : {}'.format(neighbor_feature_index.shape)) # Edge_index
print('Neighbor Features Shape : {}'.format(neighbor_features.shape))           # Edge_weights : Distance (Scalar)
print('Neighbor Coordinates Shape : {}'.format(neighbor_coordinates.shape))     # Neighbor Node Coordinates

1. **유닛셀 안의 각 기준노드(0번부터 (N-1)번 원자까지) 각각에 대해 탐색영역 안에서 선택된 이웃노드들의 인덱스 (유닛셀 경계를 넘어 중복된 종류의 노드가 연결될 수 있다. --> 그러므로 Multi-Graph)**
2. **Multi-Graph이므로 보통의 Square Adjacency Matrix [N, N]과는 다르다**.
3. **여기서는 일반적으로 Edge_index를 [N, M]으로 봐야 한다. M은 유닛셀 경계를 넘어 연결되어 있는 이웃노드(또는 Edge)의 수로 제한이 없지만, 여기서는 M = max number of neighbors로 주어진다**.
4. **또한 Edge_index는 일반적인 0과 1의 Binary Matrix가 아닌, 정수 인덱스 행렬 (source to destination)으로 주어진다**.





#### Gaussian Expansion

* For Edge Attributes Matrix



In [ ]:
class GaussianDistance(object):
    def __init__(self, dmin, dmax, step, var = None):
        assert dmin < dmax
        assert dmax - dmin > step
        self.filter = np.arange(dmin, dmax + step, step)
        if var is None:
            var = step
        self.var = var

    def expand(self, distances):
        return np.exp( -(distances[..., np.newaxis] - self.filter)**2 / self.var**2) # Gaussian Expansion

Gaussian = GaussianDistance(dmin = 0, dmax = radius, step = 0.2)
print(Gaussian)

# Distance(scalar, edge weight) --> Gaussian Expanded array
# Gaussian Basis로 확장된 이 거리행렬이 실제 결정구조 내의 모든 Edge에 대한 Edge Attributes로 작용한다.
neighbor_features_expanded = Gaussian.expand(neighbor_features) # [N, M] --> [N, M, num_filter]
edge_attr_dim = neighbor_features_expanded.shape[-1]
print('Neighbor Features Expanded by Gaussian Basis : {}'.format(neighbor_features_expanded.shape))
print('Edge Attr Dimension : {}'.format(edge_attr_dim))

#### 데이터 정리

* **이제 Node Attributes와 Edge Attributes, Edge Index를 모두 알고 있으므로 Graph Neural Network를 위한 데이터 준비가 가능하다**.



In [ ]:
import networkx as nx
G = nx.MultiGraph() # Undirected Multi-Graph

node_labels = []
N = len(crystal)                      # Number of Atoms in the Single Crystal
for i in range(N):                    # Crystal을 구성하는 원자의 수만큼 반복
    element = crystal[i].specie.name  # Crystal의 i번째 원자의 specie에 name으로 접근
    node_labels.append(element)       # Element name을 Node label로 정리
    G.add_node(element)
# ---------------------------------- Multi-Edge Test ----------------------------------

# --------------------------------------------------------------------------------------
print('Node Labels : {}'.format(node_labels))

In [ ]:
import torch
# ================================ Graph Information ================================
# Node Informations
node_labels = node_labels
node_attributes = torch.Tensor(atomic_attributes)           # [N, Embedding Dimension (92)]

# Edge Informations
edge_index      = torch.Tensor(neighbor_feature_index)      # Edge Index [N, M]
edge_attributes = torch.Tensor(neighbor_features_expanded)  # Edge Attributes [N, M, num_filter]
edge_distances  = torch.Tensor(neighbor_features)           # Edge Distance Weight [N, M] 
neighbor_coords = torch.Tensor(neighbor_coordinates)        # Neighbor Coordinates [N, M, 3]

# Target Property and CIF ID
target_property = target

print('=============================== Node Informations ===============================')
print('Node Labels : {}'.format(node_labels))
print('Node Attributes Shape : {}'.format(node_attributes.shape))
print()

print('=============================== Edge Informations ===============================')
print('Edge Index Shape : {}'.format(edge_index.shape))
print('Edge Attributes Shape : {}'.format(edge_attributes.shape))
print('Edge Distances Shape : {}'.format(edge_distances.shape))
print('Neighbor Coordinates : {}'.format(neighbor_coords.shape))
print()

print('========================== Target Property and CIF ID ===========================')
print('Target Property : {}'.format(target_property))
print('Index : {}'.format(idx)) # idx 지정하면 이후의 모든 sample 데이터를 정리하기 가능!!

#### Edge_index[N, M] --> Edge_index [2, num_edges (source to destination)] 변환하기

* **Multi-Graph가 되도록 허용**
* **즉, 동일한 원자번호는 서로 다른 좌표에 있을지라도 동일한 노드가 되어야 함!**



In [ ]:
edge_index = edge_index.long() # To integer 64bit
print('Edge Index Shape : {}'.format(edge_index.shape))
print(edge_index)

In [ ]:
N = edge_index.shape[0]
M = edge_index.shape[1]
print('N: {}, M: {}'.format(N, M))


new_edge_index = []
for i in range(N):          # 각각의 행에 대해
    central_node_index = i  # 중심노드 인덱스 i에 대해
    for j in range(M):
        neighbor_node_index = edge_index[i][j]
        index_tuple = (central_node_index, neighbor_node_index.item()) # Neighbor node index는 tensor이므로 .item()으로 값 추출
        new_edge_index.append(index_tuple)
new_edge_index = torch.Tensor(new_edge_index)
print(new_edge_index.shape)


# -----------------------------------------------------
undirected_edge_index = []
for i in range(len(new_edge_index)):
    source_idx, target_idx = new_edge_index[i]
    inverse_edge = torch.Tensor([target_idx.item(), source_idx.item()])
    out = torch.cat([new_edge_index[i], inverse_edge]).view(-1, 2).long().numpy()
    undirected_edge_index.append(out)

undirected_edge_index = np.array(undirected_edge_index)
undirected_edge_index = torch.Tensor(undirected_edge_index).view(-1, 2)
undirected_edge_index = undirected_edge_index.t()
print(undirected_edge_index.shape)

#### 위에서 재정의된 new_edge_index에 대응하여 edge_attributes 행렬도 재정의되어야 한다.

* **[N, M, num_filter] --> [N * M, num_filter]**
* **Edge의 번호가 잘 매칭되어야 한다!**



In [ ]:
# Check
print(edge_attributes.shape)
new_edge_attributes = edge_attributes.view(-1, edge_attr_dim)
print(new_edge_attributes.shape)

# ------------------------------------------------
undirected_edge_attributes = []
for i in range(len(new_edge_attributes)):
    temp = new_edge_attributes[i].clone()
    out = torch.cat([new_edge_attributes[i], temp]).view(-1, edge_attr_dim).numpy()
    undirected_edge_attributes.append(out)
    
undirected_edge_attributes = np.array(undirected_edge_attributes)
undirected_edge_attributes = torch.Tensor(undirected_edge_attributes).view(-1, edge_attr_dim)
print(undirected_edge_attributes.shape)

In [ ]:
from torch_geometric.data import Data
from torch_geometric.utils.undirected import to_undirected

In [ ]:
x = node_attributes.clone()
edge_index = undirected_edge_index.long().clone()
edge_attr  = undirected_edge_attributes.double()
y = torch.tensor([target_property])

data = Data(x = x, edge_index = edge_index, edge_attr = edge_attr, y = y, pos = None,
            node_labels = node_labels)

print(data)
print('=======================================================')

# Gather some statistics about the graph.
print('Number of nodes : {}'.format(data.num_nodes))
print('Number of edges : {}'.format(data.num_edges))
print('Average node degree : {:.2f}'.format(data.num_edges / data.num_nodes))

# Methods
print('Has isolated nodes : {}'.format(data.has_isolated_nodes())) # 모든 중심원자들은 동일한 수 M개의 이웃을 가지므로 고립된 노드는 있을 수 없음
print('Has self-loops : {}'.format(data.has_self_loops())) # 유닛셀 경계 너머 자기자신과 같은 인덱스의 원자로 이어질 수 있으므로 self loop는 가능
print('Is undirected : {}'.format(data.is_undirected())) # 방향성이 없는 그래프 구조여야 하는데 왜 안 되지??

print('Node Labels : {}'.format(data.node_labels))

#### 결정 그래프 시각화

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.utils import to_networkx
from sklearn.manifold import TSNE

def visualize_graph(G): # 그래프를 시각화하는 함수
    plt.figure(figsize = (7, 7))
    plt.xticks([])
    plt.yticks([])
    nx.draw_networkx(G, pos = nx.spring_layout(G, seed = 42), with_labels = False, node_color = None, cmap = "Set2")
    plt.show()

G = to_networkx(data, to_undirected=True)
visualize_graph(G) # PyG Data 객체 시각화

## =============== CustomDataset ===============

* 개별 데이터에 대해 PyG 형식으로 전처리가 가능했다면
* 전체 데이터에 대해 전처리하여 index 지정 방식으로 가져올 수 있는 커스텀 데이터셋 클래스를 작성해야 한다.



In [ ]:
from torch.utils.data import Dataset
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader # 그래프 배치를 자동화해줌

class CustomDataset(Dataset):

    def __init__(self, new_id_prop_data = new_id_prop_data, embedding_dict = embedding_dict,
                 cutoff_radius = 9, M = 12):
        super(CustomDataset, self).__init__()
        self.new_id_prop_data = new_id_prop_data
        self.embedding_dict   = embedding_dict
        self.cutoff_radius    = cutoff_radius
        self.M = M

    def __len__(self):
        return len(self.new_id_prop_data)

    def __getitem__(self, index):

        sample = self.new_id_prop_data[index] # idx에 의해 Structure / Target Property 추출
        if sample == []:
            return 0
        else:
            crystal = sample[0]  # Sample 데이터의 Structure 정보 접근
            target  = sample[1]  # Sample 데이터의 Target Property 정보 접근

            node_labels = []
            N = len(crystal)                      # Number of Atoms in the Single Crystal
            for i in range(N):                    # Crystal을 구성하는 원자의 수만큼 반복
                element = crystal[i].specie.name  # Crystal의 i번째 원자의 specie에 name으로 접근
                node_labels.append(element)       # Element name을 Node label로 정리

            atomic_attributes = [self.embedding_dict[crystal[i].specie.number] for i in range(len(crystal))] # Crystal의 i번째 element number로 접근하여 임베딩벡터 맵핑
            atomic_attributes = np.array(atomic_attributes)

            # 유닛셀 안의 각 원자에 대해 반경 안에 존재하는 이웃노드들의 위치정보
            all_neighbors = crystal.get_all_neighbors(r = self.cutoff_radius, include_index = True)
            # x[1] : 유닛셀 안의 중심노드와 이웃노드 사이의 거리를 기반으로 정렬
            all_neighbors_sorted = [sorted(neighbor, key = lambda x : x[1]) for neighbor in all_neighbors]


            # 이웃노드들의 인덱스 행렬과 특성정보
            neighbor_feature_index = []
            neighbor_features = []
            neighbor_coordinates = []
            max_number_of_neighbors = self.M  # M

            for neighbor in all_neighbors_sorted:
                neighbor_feature_index.append(list(map(lambda x : x[2], neighbor[:max_number_of_neighbors])))             # x[2] : 이웃원자 인덱스
                neighbor_features.append(list(map(lambda x : x[1], neighbor[:max_number_of_neighbors])))                  # x[1] : 중심원자와 이웃원자 사이 거리
                neighbor_coordinates.append(np.array(list(map(lambda x : x.coords, neighbor[:max_number_of_neighbors])))) # 이웃원자 좌표

            # Convert to Numpy Arrays
            neighbor_feature_index = np.array(neighbor_feature_index)
            neighbor_features = np.array(neighbor_features)
            neighbor_coordinates = np.array(neighbor_coordinates)

            # Distance(scalar, Edge Weight) --> Gaussian Function array
            # Gaussian Basis로 확장된 이 거리행렬이 실제 결정구조 내의 모든 Edge에 대한 Edge Attributes로 작용한다.
            neighbor_features_expanded = Gaussian.expand(neighbor_features) # [N, M] --> [N, M, num_filter]

            # --------------------------- Graph Informations ------------------------------
            # Node Informations
            # node_labels = node_labels
            node_attributes = torch.Tensor(atomic_attributes)

            # Edge Informations
            edge_index      = torch.Tensor(neighbor_feature_index)
            edge_attributes = torch.Tensor(neighbor_features_expanded)
            edge_distance   = torch.Tensor(neighbor_features)

            # Target Property and CIF ID
            target_property = torch.Tensor([target])

            # PyG 그래프 데이터로 변환
            # --------------------------------------------------
            N = edge_index.shape[0]
            M = edge_index.shape[1]
            new_edge_index = []
            for i in range(N): # 각각의 행에 대해
                central_node_index = i # 중심노드 인덱스 i에 대해
                for j in range(M):
                    neighbor_node_index = edge_index[i][j]
                    index_tuple = (central_node_index, neighbor_node_index.item()) # Neighbor node index는 tensor이므로 .item()으로 값 추출
                    new_edge_index.append(index_tuple)
            new_edge_index = torch.Tensor(new_edge_index)
            
            # ------------------ undirected edge index ------------------
            undirected_edge_index = []
            for i in range(len(new_edge_index)):
                source_idx, target_idx = new_edge_index[i]
                inverse_edge = torch.Tensor([target_idx.item(), source_idx.item()])
                out = torch.cat([new_edge_index[i], inverse_edge]).view(-1, 2).long().numpy()
                undirected_edge_index.append(out)

            undirected_edge_index = np.array(undirected_edge_index)
            undirected_edge_index = torch.Tensor(undirected_edge_index).view(-1, 2)
            undirected_edge_index = undirected_edge_index.t()

            # ------------------ undirected edge attr ------------------
            new_edge_attributes = edge_attributes.view(-1, edge_attr_dim)
            undirected_edge_attributes = []
            for i in range(len(new_edge_attributes)):
                temp = new_edge_attributes[i].clone()
                out = torch.cat([new_edge_attributes[i], temp]).view(-1, edge_attr_dim).numpy()
                undirected_edge_attributes.append(out)
                
            undirected_edge_attributes = np.array(undirected_edge_attributes)
            undirected_edge_attributes = torch.Tensor(undirected_edge_attributes).view(-1, edge_attr_dim)

            # 4가지 그래프 속성 준비
            x = node_attributes.clone()
            edge_index = undirected_edge_index.long().clone()
            edge_attr = undirected_edge_attributes.clone().float()
            y = torch.tensor([target])

            data = Data(x = x, edge_index = edge_index, edge_attr = edge_attr, y = y, pos = None,
                        node_labels = node_labels)
            return data

custom_dataset = CustomDataset()
print(custom_dataset)
print('Number of Total Crystal Dataset : {}'.format(len(custom_dataset)))

## Normalizer / mae

* 샘플데이터를 기반으로 Normalizer의 mean, std 값을 추출하여 이후 다른 모든 데이터에 대해 norm, denorm method를 적용



In [ ]:
from random import sample 

# 2000개의 샘플데이터로 normalize의 mean, std 생성하여 기록 --> 추후 새로운 물성추론값을 norm/denorm 할 때 사용
sample_target_list = [custom_dataset[i].y for i in sample(population = range(len(custom_dataset)), k = 2000)]
print(sample_target_list)

In [ ]:
def mae(prediction, target):
    """
    Computes the mean absolute error between prediction and target
    prediction : torch.Tensor (N, 1)
    target : torch.Tensor (N, 1)
    """
    return torch.mean(torch.abs(target - prediction))

test_tar = torch.tensor([2, 4, 6, 8]).float()
test_pre = torch.tensor([10, 10, 10, 10]).float()
print(mae(test_pre, test_tar)) # 6

tensor(5.)


In [ ]:
class Normalizer(object): 
    """ Normalize a Tensor and restore it later. """
    def __init__(self, tensor):
        """ tensor is taken as a sample to calculate the mean and std """
        self.mean = torch.mean(tensor)
        self.std  = torch.std(tensor)

    def norm(self, tensor):          # Normalize method
        return (tensor - self.mean) / self.std

    def denorm(self, normed_tensor): # Denormalize method
        return normed_tensor * self.std + self.mean

    def state_dict(self):
        return {'mean' : self.mean, 'std' : self.std}
    
    def load_state_dict(self, state_dict):
        self.mean = state_dict['mean']
        self.std  = state_dict['std']

normalizer = Normalizer(tensor = torch.Tensor(sample_target_list)) # normalizer에 샘플 데이터들의 mean / std가 기억된다.
print('Sample Mean : {}'.format(normalizer.mean))
print('Sample Std  : {}'.format(normalizer.std))

Sample Mean : 3.0536046028137207
Sample Std  : 0.22289882600307465


In [ ]:
num_trains = 110000
num_valids = 20000

train_dataset = CustomDataset(new_id_prop_data = new_id_prop_data[:num_trains])
valid_dataset = CustomDataset(new_id_prop_data = new_id_prop_data[num_trains : num_trains + num_valids])
test_dataset  = CustomDataset(new_id_prop_data = new_id_prop_data[num_trains + num_valids:])

train_loader = DataLoader(train_dataset, batch_size = 256, shuffle = True,  pin_memory = True, num_workers = 4)
valid_loader = DataLoader(valid_dataset, batch_size = 256, shuffle = False, pin_memory = True)
test_loader  = DataLoader(test_dataset,  batch_size = 256, shuffle = False)

print('Number of Train Dataset : {}'.format(len(train_loader.dataset)))
print('Number of Valid Dataset : {}'.format(len(valid_loader.dataset)))
print('Number of Test Dataset : {}'.format(len(test_loader.dataset)))

sample = next(iter(train_loader))
print('Sample : {}'.format(sample))  # Check Batch Sample

Number of Train Dataset : 110000
Number of Valid Dataset : 20000
Number of Test Dataset : 12701
Sample : DataBatch(x=[8323, 92], edge_index=[2, 199752], edge_attr=[199752, 41], y=[256], node_labels=[256], batch=[8323], ptr=[257])


## ============== CGCNN PyG Version ==============



In [ ]:
import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, CGConv # CGCNN CONV
from torch_geometric.nn import global_mean_pool

from tqdm import tqdm
import shutil
import time

In [ ]:
class CGCNN(torch.nn.Module):

    def __init__(self, in_channels = 92, edge_attr_dim = edge_attr_dim):
        super().__init__()
        self.conv1 = CGConv(channels = in_channels, dim = edge_attr_dim, batch_norm = True) # Node vector 차원 유지
        self.conv2 = CGConv(channels = in_channels, dim = edge_attr_dim, batch_norm = True) # BatchNorm 내부에 존재
        self.conv3 = CGConv(channels = in_channels, dim = edge_attr_dim, batch_norm = True)
        self.lin4 = Linear(in_channels, 1) # Regression

    def forward(self, x, edge_index, edge_attr, batch):
        x = self.conv1(x, edge_index, edge_attr) # edge_attr까지 사용되는지 본다.
        x = F.leaky_relu(x)
        x = F.dropout(x, p = 0.5, training = self.training)

        x = self.conv2(x, edge_index, edge_attr)
        x = F.leaky_relu(x)
        x = F.dropout(x, p = 0.5, training = self.training)

        x = self.conv3(x, edge_index, edge_attr) # Updated Node Embedding
        x = F.leaky_relu(x)
        h = F.dropout(x, p = 0.5, training = self.training)

        x = global_mean_pool(h, batch)           # [batch_size, hidden_channels]
        x = F.dropout(x, p = 0.5, training = self.training) 
        out = self.lin4(x) 
        return out.view(-1), h                   # Node Embedding 함께 출력

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CGCNN().to(device)
sample = sample.to(device)
out, _ = model(sample.x, sample.edge_index, sample.edge_attr, sample.batch)
print(out.shape, sample.y.shape) # out과 sample.y는 batch에 의한 shape가 조금 다름.

torch.Size([256]) torch.Size([256])


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CGCNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001, weight_decay = 0.001)
criterion = torch.nn.MSELoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer = optimizer, mode = 'min',
                                                       patience = 3, verbose = True)
print('Model : {}'.format(model))
print('Criterion : {}'.format(criterion))
print('Optimizer : {}'.format(optimizer))
print('Scheduler : {}'.format(scheduler))

Model : CGCNN(
  (conv1): CGConv(92, dim=41)
  (conv2): CGConv(92, dim=41)
  (conv3): CGConv(92, dim=41)
  (lin4): Linear(in_features=92, out_features=1, bias=True)
)
Criterion : MSELoss()
Optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0.001
)
Scheduler : <torch.optim.lr_scheduler.ReduceLROnPlateau object at 0x7fb74be62f50>


### Training / Validation / Saving

[해결]

* DataLoader의 경우 pin memory 와 num workers 옵션으로도 속도가속이 빠르진 않다. 그러나 미리 모든 데이터를 메모리에 올려 사용하지 말 것!! 
* Target Property에 대해 평균 0, 표준편차 1의 분포로 Normalization 완료! (혹시 더 좋은 정규화 방법 있는지 검색) 
* LR Scheduler를 사용하여 LR Decay 적용 완료.
* Best Performance Model Save 구현
* Early Stopping 구현
* 모델 훈련시간 측정 구현

[남은 것]

* **다른 효율적인 regression cost function은 없는지 찾아보기**
* 효율적인 LR Scheduler 조사하기
* **Tensor Board** 연동하기 or 학습과정 시각화하기
* **효율적인 graph pooling 조사하기** mean pooling은 별로 안좋은듯??

[학습최적화]

* batch size
* optimizer : Adam, SGD, RMSProp 등
* learning rate, weight decay
* dropout, batchnorm

In [ ]:
model_path = './01.[PyG]CGCNN-HalidePerovskite-EarlyStopping.pth' 
num_epochs = 60
best_mae_error = 100
EarlyStopping_Patience = 8
avg_train_losses = [] # To plot loss graph
avg_valid_losses = []

In [ ]:
earlystopping_count = 0
start_time = time.time() # Start Time
for epoch in range(num_epochs):
    # ------------------------------------ Training -----------------------------------------------
    model.train()
    training_loss = 0
    for batch_data in tqdm(train_loader):
        optimizer.zero_grad()
        batch_data = batch_data.to(device)
        x, edge_index, edge_attr, batch = batch_data.x, batch_data.edge_index, batch_data.edge_attr, batch_data.batch

        target = batch_data.y                          # Pure Target
        normed_target = normalizer.norm(target)        # Normalized Target
    
        out, h = model(x, edge_index, edge_attr, batch)   # Single Performance
        assert out.shape == normed_target.shape, 'Shapes are different!!'
        
        loss = criterion(out, normed_target)  # normed_target과 비교하여 학습되어야 한다.
        training_loss += loss.item()                  
        loss.backward()   # Back-propagation
        optimizer.step()  # Update parameters
    average_training_loss = training_loss / len(train_loader.dataset)
    avg_train_losses.append(average_training_loss)
    # --------------------------------------------------------------------------------------------
    # ----------------------------------- Validation ---------------------------------------------
    model.eval()
    val_loss = 0
    for batch_data in tqdm(valid_loader):
        batch_data = batch_data.to(device)
        x, edge_index, edge_attr, batch = batch_data.x, batch_data.edge_index, batch_data.edge_attr, batch_data.batch
        target = batch_data.y  # Pure Target
        
        pred, h = model(x, edge_index, edge_attr, batch)  # Prediction in normed scale
        assert pred.shape == target.shape, 'Shapes are different!!'
        denormed_pred = normalizer.denorm(pred)        # denormed prediction

        val_mae = mae(denormed_pred, target)  # denorm 스케일에서 Pure target과 pred가 비교되어야 한다.
        val_loss += val_mae.item()                    
    average_validation_loss = val_loss / len(valid_loader.dataset)
    avg_valid_losses.append(average_validation_loss)
    
    # ------------------------------ Print / LR Schedule ------------------------------------------
    print('Epoch: [{:02d}/{:02d}], Training MSE Loss: {:.6f}, Validation MAE Loss: {:.6f}'.format(
        epoch + 1, num_epochs, average_training_loss, average_validation_loss))
    
    scheduler.step(metrics = average_validation_loss)     # Average val loss 기준으로 Learning Rate 조정
    
    # ---------------------------- Save Best Model and Early Stopping ----------------------------
    is_best = average_validation_loss < best_mae_error             # True or False
    best_mae_error = min(average_validation_loss, best_mae_error)  # minimum value between them --> new best mae error

    torch.save(model.state_dict(), 'checkpoint.pth.tar')  # Epoch마다 모델의 state dict를 checkpoint로 모두 저장
    if is_best:                                           # 최고성능 모델이면
        shutil.copyfile('checkpoint.pth.tar', model_path) # checkpoint를 best model로 복사 및 저장
        print('Best Model is Saved When Epoch = {}'.format(epoch))
        earlystopping_count = 0                           # 향상되면 EarlyStopping Patience = 0으로 초기화 
    else:                                                 # 향상되지 않으면
        earlystopping_count += 1                          # patience count에 1을 추가

    if earlystopping_count == EarlyStopping_Patience: # patience에 도달하면
        print('Model is Early Stopped When Epoch = {}'.format(epoch))
        break                                         # 전체 학습 종료
    # ---------------------------------------------------------------------------------------------

print('\nElapsed Time : {:.4f} sec'.format(time.time() - start_time))
print('Training Completed!!')        

In [ ]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.validators.scatter.marker import SymbolValidator

px.scatter()


plt.figure(figsize = (10, 10))             
plt.title('Training Validation Loss Comparison', size = 20)

plt.plot(range(num_epochs), avg_train_losses, label = 'train', color = 'blue')
plt.scatter(range(num_epochs), avg_train_losses)
plt.plot(range(num_epochs), avg_valid_losses, label = 'valid', color = 'red')
plt.scatter(range(num_epochs), avg_valid_losses)

plt.xlabel('Epoch', size = 16)
plt.ylabel('Loss', size = 16)
plt.grid(True)
plt.legend()
plt.show()

### 텐서보드 테스트

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import numpy as np

writer = SummaryWriter()

for n_iter in range(100):
    writer.add_scalar('Loss/train', np.random.random(), n_iter)
    writer.add_scalar('Loss/test', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/train', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/test', np.random.random(), n_iter)

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=runs 

### Test / Node Embedding




In [ ]:
# --------------------------------- Model Reload -------------------------------------------------
model = CGCNN().to(device)
model.load_state_dict(torch.load(model_path))
model.eval()
# ------------------------------------------------------------------------------------------------

test_loss = 0
for batch_data in tqdm(test_loader):
    batch_data = batch_data.to(device)
    x, edge_index, edge_attr, batch = batch_data.x, batch_data.edge_index, batch_data.edge_attr, batch_data.batch
    target = batch_data.y

    pred, h = model(x, edge_index, edge_attr, batch) 
    assert pred.shape == target.shape, 'Shapes are different!!'
    denormed_pred = normalizer.denorm(pred)
    test_mae = mae(denormed_pred, target)      # Test MAE
    test_loss += test_mae.item()                    
average_test_mae = test_loss / len(test_loader.dataset) # Average Test MAE
print('Average Test MAE : {:.6f}'.format(average_test_mae))

100%|██████████| 50/50 [08:57<00:00, 10.75s/it]

Average Test MAE : 0.000098


## ============ Atomic Node TSNE Plot ==========



In [ ]:
test_sample = next(iter(test_loader)).to(device) # Test Batch Sample
print(test_sample)      # 여기서 node_labels[i]는 i번째 결정의 노드라벨집합이므로 풀어주어야 한다.
print(test_sample.ptr)  # Pointer로 single graph에 접근가능

DataBatch(x=[8119, 92], edge_index=[2, 194856], edge_attr=[194856, 41], y=[256], node_labels=[256], batch=[8119], ptr=[257])
tensor([   0,   39,   49,   59,   79,  117,  155,  193,  198,  218,  256,  261,
         299,  339,  379,  417,  455,  494,  533,  572,  611,  616,  654,  694,
         733,  743,  782,  820,  859,  898,  937,  975, 1013, 1052, 1072, 1077,
        1115, 1155, 1193, 1232, 1252, 1272, 1310, 1348, 1387, 1426, 1464, 1502,
        1541, 1579, 1618, 1656, 1694, 1704, 1742, 1747, 1785, 1825, 1864, 1903,
        1941, 1961, 2000, 2038, 2078, 2116, 2154, 2159, 2169, 2207, 2246, 2284,
        2322, 2361, 2399, 2404, 2443, 2483, 2488, 2526, 2565, 2603, 2643, 2682,
        2721, 2759, 2769, 2807, 2845, 2850, 2888, 2926, 2946, 2986, 3024, 3044,
        3082, 3120, 3130, 3168, 3206, 3244, 3283, 3321, 3360, 3365, 3403, 3441,
        3461, 3500, 3538, 3577, 3616, 3621, 3659, 3669, 3708, 3728, 3766, 3804,
        3809, 3849, 3869, 3874, 3912, 3951, 3990, 4029, 4068, 4106, 4126, 4

In [ ]:
batch_node_labels = []
for each_node_labels in test_sample.node_labels: # 각각의 결정구조 노드라벨을 추출하여
    batch_node_labels.extend(each_node_labels)   # Extend로 이어붙임

test_sample.node_labels = batch_node_labels      # Redefine
assert test_sample.x.shape[0] == len(test_sample.node_labels) # Must be "Number of Node == Number of node_labels"
print(test_sample) 

DataBatch(x=[8119, 92], edge_index=[2, 194856], edge_attr=[194856, 41], y=[256], node_labels=[8119], batch=[8119], ptr=[257])


In [ ]:
import pandas as pd
label_to_color_dict = {'Cs': 'red',
                       'Pb': 'yellow', 'Sn': 'green',
                       'Br': 'blue', 'Cl': 'violet', 'I': 'black'}

### Trained Node Embedding

* Plotly를 이용하여 겹쳐있는 부근의 원자노드를 선택추출하고
* 그로부터 노드번호와 결정구조 번호를 역추적
* 그러나 결정구조를 역추적해도 얻을 수 있는 것은 임베딩된 그래프 정보뿐,
* 여기서 **본래의 결정구조인 포인트클라우드를 아는 것은 어려워보인다.**




In [ ]:
perplexities = [2, 5, 10, 20, 30, 40, 50, 100] # Grid Search for Perplexity
out, trained_h = model(test_sample.x, test_sample.edge_index, test_sample.edge_attr, test_sample.batch) # Trained Embedding
fig, axs = plt.subplots(1, len(perplexities), figsize = (64, 8))

start_time = time.time()
for i, perplexity in tqdm(enumerate(perplexities)):

    train_tsne = TSNE(n_components = 2, perplexity = perplexity, learning_rate = 'auto', init = 'pca')
    trained_z = train_tsne.fit_transform(trained_h.detach().cpu().numpy())

    # Trained Node Embedding
    df = pd.DataFrame(data = None)
    df['label'] = test_sample.node_labels
    df['x'] = trained_z[:, 0]
    df['y'] = trained_z[:, 1]
    df['color'] = df['label'].apply(lambda x: label_to_color_dict[x])

    # 원자별 라벨 분류
    df_Cs = df[df['label'] == 'Cs']
    df_Pb = df[df['label'] == 'Pb']
    df_Sn = df[df['label'] == 'Sn']
    df_Br = df[df['label'] == 'Br']
    df_Cl = df[df['label'] == 'Cl']
    df_I  = df[df['label'] == 'I']
    df_list = [df_Cs, df_Pb, df_Sn, df_Br, df_Cl, df_I]

    ax = axs[i]
    for df, label, color in zip(df_list, label_to_color_dict.keys(), label_to_color_dict.values()):
        ax.set_title("Perplexity = %d" % perplexity, size = 20)
        ax.scatter(df['x'], df['y'], s = 70, color = color, label = label, alpha = 0.25)
        ax.axis('tight')
        ax.legend()

print('Elapsed Time : {:.2f} sec'.format(time.time() - start_time))
plt.show()

### Untrained Node Embedding



In [ ]:
perplexities = [2, 5, 10, 20, 30, 40, 50, 100] # Grid Search for Perplexity
initial_model = CGCNN().to(device) # 모델 초기화
out, untrained_h = initial_model(test_sample.x, test_sample.edge_index, test_sample.edge_attr, test_sample.batch)
fig, axs = plt.subplots(1, len(perplexities), figsize = (64, 8))

start_time = time.time()
for i, perplexity in tqdm(enumerate(perplexities)):

    untrain_tsne = TSNE(n_components = 2, perplexity = perplexity, learning_rate = 'auto', init = 'pca')
    untrained_z = untrain_tsne.fit_transform(untrained_h.detach().cpu().numpy())

    # Trained Node Embedding
    df = pd.DataFrame(data = None)
    df['label'] = test_sample.node_labels
    df['x'] = untrained_z[:, 0]
    df['y'] = untrained_z[:, 1]
    df['color'] = df['label'].apply(lambda x: label_to_color_dict[x])

    # 원자별 라벨 분류
    df_Cs = df[df['label'] == 'Cs']
    df_Pb = df[df['label'] == 'Pb']
    df_Sn = df[df['label'] == 'Sn']
    df_Br = df[df['label'] == 'Br']
    df_Cl = df[df['label'] == 'Cl']
    df_I  = df[df['label'] == 'I']
    df_list = [df_Cs, df_Pb, df_Sn, df_Br, df_Cl, df_I]

    ax = axs[i]
    for df, label, color in zip(df_list, label_to_color_dict.keys(), label_to_color_dict.values()):
        ax.set_title("Perplexity = %d" % perplexity, size = 20)
        ax.scatter(df['x'], df['y'], s = 70, color = color, label = label, alpha = 0.25)
        ax.axis('tight')
        ax.legend()

print('Elapsed Time : {:.2f} sec'.format(time.time() - start_time))
plt.show()

### Initial Node Embedding



In [ ]:
perplexities = [2, 5, 10, 20, 30, 40, 50, 100] # Grid Search for Perplexity
fig, axs = plt.subplots(1, len(perplexities), figsize = (64, 8))

start_time = time.time()
for i, perplexity in tqdm(enumerate(perplexities)):

    initial_tsne = TSNE(n_components = 2, perplexity = perplexity, learning_rate = 'auto', init = 'pca')
    initial_x = initial_tsne.fit_transform(test_sample.x.detach().cpu().numpy())

    # Trained Node Embedding
    df = pd.DataFrame(data = None)
    df['label'] = test_sample.node_labels
    df['x'] = initial_x[:, 0]
    df['y'] = initial_x[:, 1]
    df['color'] = df['label'].apply(lambda x: label_to_color_dict[x])

    # 원자별 라벨 분류
    df_Cs = df[df['label'] == 'Cs']
    df_Pb = df[df['label'] == 'Pb']
    df_Sn = df[df['label'] == 'Sn']
    df_Br = df[df['label'] == 'Br']
    df_Cl = df[df['label'] == 'Cl']
    df_I  = df[df['label'] == 'I']
    df_list = [df_Cs, df_Pb, df_Sn, df_Br, df_Cl, df_I]

    ax = axs[i]
    for df, label, color in zip(df_list, label_to_color_dict.keys(), label_to_color_dict.values()):
        ax.set_title("Perplexity = %d" % perplexity, size = 20)
        ax.scatter(df['x'], df['y'], s = 70, color = color, label = label, alpha = 0.25)
        ax.axis('tight')
        ax.legend()

print('Elapsed Time : {:.2f} sec'.format(time.time() - start_time))
plt.show()

## ============ Model Prediction / DFT 비교 ============

* Halide Perovskite (ABX3) : B 원자 좌표변경
* 2번 좌표를 대각선 방향으로 이동시키며 energy per atom 변화곡선 그리기
* 좌표이동범위는 0.01 ~ 0.03 정도로
* DFT 계산도 같이 해서 함께 비교할 것!!



In [ ]:
import pymatgen
from pymatgen.ext.matproj import MPRester
from tqdm import tqdm

# ------------------------------ 예전에 계산된 DFT 결과 ------------------------------
DFT_CsPbI3 = [-3.001358142, -3.000530674, -2.997699524, -2.991343506]
DFT_CsPbBr3 = [-3.357488076, -3.3562551480000002, -3.352473614, -3.3447681680000003]
DFT_CsPbCl3 = [-3.6798636559999998, -3.678500538, -3.673609942, -3.6637402139999997]
DFT_CsSnI3 = [-3.007006572, -3.006766266, -3.0053754919999998, -3.000954948]
DFT_CsSnBr3 = [-3.356927898, -3.356391218, -3.354186868, -3.348770438]
DFT_CsSnCl3 = [-3.668088058, -3.66733331, -3.664741496, -3.658909414]
# ----------------------------------------------------------------------------------
halide_perovskites_mp_ids = {'mp-614013'  : 'CsSnI3',
                             'mp-1070375' : 'CsSnCl3',
                             'mp-27214'   : 'CsSnBr3',
                             'mp-1069538' : 'CsPbI3',
                             'mp-23037'   : 'CsPbCl3',
                             'mp-600089'  : 'CsPbBr3'}
new_data = {}
my_API_key = 'I5DdZsmHO3er6WKz' # You can get your own API key from the homepage of material project
with MPRester(api_key = my_API_key) as m:

    for mp_id in halide_perovskites_mp_ids.keys():
        new_data[mp_id] = (m.query(criteria = {'material_id' : mp_id},
                                   properties = ['energy_per_atom', 'structure']))

#### mp-1069538 / CsPbI3

In [ ]:
# ---------- # 여기만 바꿔주면 된다!! ----------
mp_id = 'mp-1069538'   
mp_name = 'CsPbI3'
changed_atom = 'Pb'
# --------------------------------------------
energy_per_atom_list = []
Test_Material_CIF_list = []
Test_Material_Data = {}
delta = 0.01
num_samples = 4

for i in range(num_samples):

    new_data = {}
    my_API_key = 'I5DdZsmHO3er6WKz'  # You can get your own API key from the homepage of material project.

    with MPRester(my_API_key) as m:
        new_data[mp_id] = (m.query(criteria = {'material_id' : mp_id},
                                   properties = ['energy_per_atom', 'structure']))
    new_structure = new_data[mp_id][0]['structure']
    new_structure[1] = new_structure[1].specie.name, new_structure.frac_coords[1] + (delta * i)

    # 변화된 Structure --> CIF 파일로 저장
    new_structure.to(filename= 'new_{}_{}_delta_{}.cif'.format(mp_name, changed_atom, i))
    Test_Material_CIF_list.append(['new_{}_{}_delta_{}'.format(mp_name, changed_atom, i), 0]) # 물성은 추론할 것이므로 형식상 0
    Test_Material_Data['new_{}_{}_delta_{}'.format(mp_name, changed_atom, i)] = [{'structure' : new_structure,
                                                                                  'energy_per_atom' : 0}]
print('You Must Check the Contents in the "Test_Material_Data"')
print(Test_Material_Data)

In [ ]:
print('Test_Material_CIF_list Check!!')
print(Test_Material_CIF_list)
print()

idx = 2
cif_id, target = Test_Material_CIF_list[idx]
print('CIF ID : {}, Target : {}'.format(cif_id, target))

new_crystal = Test_Material_Data[cif_id][0]['structure'] # 생성된 구조정보 접근
new_crystal

In [ ]:
class NEWCustomDataset(Dataset):
    def __init__(self, new_id_prop_data = Test_Material_CIF_list,
                 embedding_dict = embedding_dict, cutoff_radius = 6, M = 8):
        super().__init__()
        self.new_id_prop_data = new_id_prop_data
        self.embedding_dict   = embedding_dict
        self.cutoff_radius    = cutoff_radius
        self.M = M

    def __len__(self):
        return len(self.new_id_prop_data)

    def __getitem__(self, index):

        cif_id, target = self.new_id_prop_data[index] # idx에 의해 Structure / Target Property 추출
        if cif_id == []:
            return 0
        else:
            crystal = Test_Material_Data[cif_id][0]['structure']
            #crystal = sample[0] # Sample 데이터의 Structure 정보 접근
            target  = sample[1]  # Sample 데이터의 Target Property 정보 접근

            atomic_attributes = [self.embedding_dict[crystal[i].specie.number] for i in range(len(crystal))] # Crystal의 i번째 element number로 접근하여 임베딩벡터 맵핑
            atomic_attributes = np.array(atomic_attributes)

            # 유닛셀 안의 각 원자에 대해 반경 안에 존재하는 이웃노드들의 위치정보
            all_neighbors = crystal.get_all_neighbors(r = self.cutoff_radius, include_index = True)
            # x[1] : 유닛셀 안의 중심노드와 이웃노드 사이의 거리를 기반으로 정렬
            all_neighbors_sorted = [sorted(neighbor, key = lambda x : x[1]) for neighbor in all_neighbors]


            # 이웃노드들의 인덱스 행렬과 특성정보
            neighbor_feature_index = []
            neighbor_features = []
            neighbor_coordinates = []
            max_number_of_neighbors = self.M  # M

            for neighbor in all_neighbors_sorted:
                neighbor_feature_index.append(list(map(lambda x : x[2], neighbor[:max_number_of_neighbors])))             # x[2] : 이웃원자 인덱스
                neighbor_features.append(list(map(lambda x : x[1], neighbor[:max_number_of_neighbors])))                  # x[1] : 중심원자와 이웃원자 사이 거리
                neighbor_coordinates.append(np.array(list(map(lambda x : x.coords, neighbor[:max_number_of_neighbors])))) # 이웃원자 좌표

            # Convert to Numpy Arrays
            neighbor_feature_index = np.array(neighbor_feature_index)
            neighbor_features = np.array(neighbor_features)
            neighbor_coordinates = np.array(neighbor_coordinates)

            # Distance(scalar, Edge Weight) --> Gaussian Function array
            # Gaussian Basis로 확장된 이 거리행렬이 실제 결정구조 내의 모든 Edge에 대한 Edge Attributes로 작용한다.
            neighbor_features_expanded = Gaussian.expand(neighbor_features) # [N, M] --> [N, M, num_filter]

            # --------------------------- Graph Informations ------------------------------
            # Node Informations
            # node_labels = node_labels
            node_attributes = torch.Tensor(atomic_attributes)

            # Edge Informations
            edge_index      = torch.Tensor(neighbor_feature_index)
            edge_attributes = torch.Tensor(neighbor_features_expanded)
            edge_distance   = torch.Tensor(neighbor_features)

            # Target Property and CIF ID
            target_property = torch.Tensor([0])

            # PyG 그래프 데이터로 변환
            # --------------------------------------------------
            N = edge_index.shape[0]
            M = edge_index.shape[1]
            new_edge_index = []
            for i in range(N): # 각각의 행에 대해
                central_node_index = i # 중심노드 인덱스 i에 대해
                for j in range(M):
                    neighbor_node_index = edge_index[i][j]
                    index_tuple = (central_node_index, neighbor_node_index.item()) # Neighbor node index는 tensor이므로 .item()으로 값 추출
                    new_edge_index.append(index_tuple)
            new_edge_index = torch.Tensor(new_edge_index)
            
            # ------------------ undirected edge index ------------------
            undirected_edge_index = []
            for i in range(len(new_edge_index)):
                source_idx, target_idx = new_edge_index[i]
                inverse_edge = torch.Tensor([target_idx.item(), source_idx.item()])
                out = torch.cat([new_edge_index[i], inverse_edge]).view(-1, 2).long().numpy()
                undirected_edge_index.append(out)

            undirected_edge_index = np.array(undirected_edge_index)
            undirected_edge_index = torch.Tensor(undirected_edge_index).view(-1, 2)
            undirected_edge_index = undirected_edge_index.t()

            # ------------------ undirected edge attr ------------------
            new_edge_attributes = edge_attributes.view(-1, edge_attr_dim)
            undirected_edge_attributes = []
            for i in range(len(new_edge_attributes)):
                temp = new_edge_attributes[i].clone()
                out = torch.cat([new_edge_attributes[i], temp]).view(-1, edge_attr_dim).numpy()
                undirected_edge_attributes.append(out)
                
            undirected_edge_attributes = np.array(undirected_edge_attributes)
            undirected_edge_attributes = torch.Tensor(undirected_edge_attributes).view(-1, edge_attr_dim)

            # 4가지 그래프 속성 준비
            x = node_attributes.clone()
            edge_index = undirected_edge_index.long().clone()
            edge_attr = undirected_edge_attributes.clone().float()
            y = torch.tensor([0])

            data = Data(x = x, edge_index = edge_index, edge_attr = edge_attr, y = y, pos = None)
            return data

new_custom_dataset = NEWCustomDataset()
new_custom_loader = DataLoader(new_custom_dataset, batch_size = 1, shuffle = False)
print(new_custom_dataset)
print('Number of Total Crystal Dataset : {}'.format(len(new_custom_dataset)))
new_sample = next(iter(new_custom_loader))

In [ ]:
# 필요하다면 여기서 모델 Reload 가능!

average_new_test_loss = 0
normed_perovskite_preds = []
denorm_perovskite_preds = []

for batch_data in new_custom_loader:
    batch_data = batch_data.to(device)
    x, edge_index, edge_attr, batch = batch_data.x, batch_data.edge_index, batch_data.edge_attr, batch_data.batch
    target = batch_data.y                           # Pure Target

    pred, h = model(x, edge_index, edge_attr, batch)  # Prediction in normed scale
    normed_perovskite_preds.append(pred)

    assert pred.view(-1).shape == target.shape, 'Shapes are different!!'
    denormed_pred = normalizer.denorm(pred)      # denormed prediction
    denorm_perovskite_preds.append(denormed_pred)

    #new_test_mae = mae(denormed_pred.view(-1), target) # target 값은 0으로 맞춰두어서 의미없고, 실제로는 DFT 계산결과와 비교한다.
    #new_test_loss += new_test_mae.item()  

# average_new_test_loss = new_test_loss / len(new_custom_loader.dataset)
# print('Average Test MAE : {:.6f}'.format(average_new_test_loss))

print(normed_perovskite_preds)
print(denorm_perovskite_preds)
normed_perov_preds = torch.Tensor(normed_perovskite_preds)
denorm_perov_preds = torch.Tensor(denorm_perovskite_preds)

plt.title(mp_id + ' ' + halide_perovskites_mp_ids[mp_id])
plt.plot(list(range(len(denorm_perov_preds))), denorm_perov_preds.cpu().numpy(), label = 'Predictions')
plt.scatter(list(range(len(denorm_perov_preds))), denorm_perov_preds.cpu().numpy())
plt.grid(True)
plt.legend()
plt.show()

# ---------------------- 여기만 DFT 리스트 바꾸면 됨!! -------------------------
plt.title(mp_id + ' ' + halide_perovskites_mp_ids[mp_id])
plt.plot(list(range(len(DFT_CsPbI3))), DFT_CsPbI3, label = 'DFT') # DFT 결과 음수이므로 절대값!!
plt.scatter(list(range(len(DFT_CsPbI3))), DFT_CsPbI3)
plt.grid(True)
plt.legend()
plt.show()
# ----------------------------------------------------------------------------

#### mp-600089 / CsPbBr3



In [ ]:
# ---------- # 여기만 바꿔주면 된다!! ----------
mp_id = 'mp-600089'   
mp_name = 'CsPbBr3'
changed_atom = 'Pb'
# --------------------------------------------
energy_per_atom_list = []
Test_Material_CIF_list = []
Test_Material_Data = {}
delta = 0.01
num_samples = 4

for i in range(num_samples):

    new_data = {}
    my_API_key = 'I5DdZsmHO3er6WKz'  # You can get your own API key from the homepage of material project.

    with MPRester(my_API_key) as m:
        new_data[mp_id] = (m.query(criteria = {'material_id' : mp_id},
                                   properties = ['energy_per_atom', 'structure']))
    new_structure = new_data[mp_id][0]['structure']
    new_structure[1] = new_structure[1].specie.name, new_structure.frac_coords[1] + (delta * i)

    # 변화된 Structure --> CIF 파일로 저장
    new_structure.to(filename= 'new_{}_{}_delta_{}.cif'.format(mp_name, changed_atom, i))
    Test_Material_CIF_list.append(['new_{}_{}_delta_{}'.format(mp_name, changed_atom, i), 0]) # 물성은 추론할 것이므로 형식상 0
    Test_Material_Data['new_{}_{}_delta_{}'.format(mp_name, changed_atom, i)] = [{'structure' : new_structure,
                                                                                  'energy_per_atom' : 0}]
print('You Must Check the Contents in the "Test_Material_Data"')
print(Test_Material_Data)

In [ ]:
print('Test_Material_CIF_list Check!!')
print(Test_Material_CIF_list)
print()

idx = 2
cif_id, target = Test_Material_CIF_list[idx]
print('CIF ID : {}, Target : {}'.format(cif_id, target))

new_crystal = Test_Material_Data[cif_id][0]['structure'] # 생성된 구조정보 접근
new_crystal

In [ ]:
class NEWCustomDataset(Dataset):
    def __init__(self, new_id_prop_data = Test_Material_CIF_list,
                 embedding_dict = embedding_dict, cutoff_radius = 6, M = 8):
        super().__init__()
        self.new_id_prop_data = new_id_prop_data
        self.embedding_dict   = embedding_dict
        self.cutoff_radius    = cutoff_radius
        self.M = M

    def __len__(self):
        return len(self.new_id_prop_data)

    def __getitem__(self, index):

        cif_id, target = self.new_id_prop_data[index] # idx에 의해 Structure / Target Property 추출
        if cif_id == []:
            return 0
        else:
            crystal = Test_Material_Data[cif_id][0]['structure']
            #crystal = sample[0] # Sample 데이터의 Structure 정보 접근
            target  = sample[1]  # Sample 데이터의 Target Property 정보 접근

            atomic_attributes = [self.embedding_dict[crystal[i].specie.number] for i in range(len(crystal))] # Crystal의 i번째 element number로 접근하여 임베딩벡터 맵핑
            atomic_attributes = np.array(atomic_attributes)

            # 유닛셀 안의 각 원자에 대해 반경 안에 존재하는 이웃노드들의 위치정보
            all_neighbors = crystal.get_all_neighbors(r = self.cutoff_radius, include_index = True)
            # x[1] : 유닛셀 안의 중심노드와 이웃노드 사이의 거리를 기반으로 정렬
            all_neighbors_sorted = [sorted(neighbor, key = lambda x : x[1]) for neighbor in all_neighbors]


            # 이웃노드들의 인덱스 행렬과 특성정보
            neighbor_feature_index = []
            neighbor_features = []
            neighbor_coordinates = []
            max_number_of_neighbors = self.M  # M

            for neighbor in all_neighbors_sorted:
                neighbor_feature_index.append(list(map(lambda x : x[2], neighbor[:max_number_of_neighbors])))             # x[2] : 이웃원자 인덱스
                neighbor_features.append(list(map(lambda x : x[1], neighbor[:max_number_of_neighbors])))                  # x[1] : 중심원자와 이웃원자 사이 거리
                neighbor_coordinates.append(np.array(list(map(lambda x : x.coords, neighbor[:max_number_of_neighbors])))) # 이웃원자 좌표

            # Convert to Numpy Arrays
            neighbor_feature_index = np.array(neighbor_feature_index)
            neighbor_features = np.array(neighbor_features)
            neighbor_coordinates = np.array(neighbor_coordinates)

            # Distance(scalar, Edge Weight) --> Gaussian Function array
            # Gaussian Basis로 확장된 이 거리행렬이 실제 결정구조 내의 모든 Edge에 대한 Edge Attributes로 작용한다.
            neighbor_features_expanded = Gaussian.expand(neighbor_features) # [N, M] --> [N, M, num_filter]

            # --------------------------- Graph Informations ------------------------------
            # Node Informations
            # node_labels = node_labels
            node_attributes = torch.Tensor(atomic_attributes)

            # Edge Informations
            edge_index      = torch.Tensor(neighbor_feature_index)
            edge_attributes = torch.Tensor(neighbor_features_expanded)
            edge_distance   = torch.Tensor(neighbor_features)

            # Target Property and CIF ID
            target_property = torch.Tensor([0])

            # PyG 그래프 데이터로 변환
            # --------------------------------------------------
            N = edge_index.shape[0]
            M = edge_index.shape[1]
            new_edge_index = []
            for i in range(N): # 각각의 행에 대해
                central_node_index = i # 중심노드 인덱스 i에 대해
                for j in range(M):
                    neighbor_node_index = edge_index[i][j]
                    index_tuple = (central_node_index, neighbor_node_index.item()) # Neighbor node index는 tensor이므로 .item()으로 값 추출
                    new_edge_index.append(index_tuple)
            new_edge_index = torch.Tensor(new_edge_index)
            
            # ------------------ undirected edge index ------------------
            undirected_edge_index = []
            for i in range(len(new_edge_index)):
                source_idx, target_idx = new_edge_index[i]
                inverse_edge = torch.Tensor([target_idx.item(), source_idx.item()])
                out = torch.cat([new_edge_index[i], inverse_edge]).view(-1, 2).long().numpy()
                undirected_edge_index.append(out)

            undirected_edge_index = np.array(undirected_edge_index)
            undirected_edge_index = torch.Tensor(undirected_edge_index).view(-1, 2)
            undirected_edge_index = undirected_edge_index.t()

            # ------------------ undirected edge attr ------------------
            new_edge_attributes = edge_attributes.view(-1, edge_attr_dim)
            undirected_edge_attributes = []
            for i in range(len(new_edge_attributes)):
                temp = new_edge_attributes[i].clone()
                out = torch.cat([new_edge_attributes[i], temp]).view(-1, edge_attr_dim).numpy()
                undirected_edge_attributes.append(out)
                
            undirected_edge_attributes = np.array(undirected_edge_attributes)
            undirected_edge_attributes = torch.Tensor(undirected_edge_attributes).view(-1, edge_attr_dim)

            # 4가지 그래프 속성 준비
            x = node_attributes.clone()
            edge_index = undirected_edge_index.long().clone()
            edge_attr = undirected_edge_attributes.clone().float()
            y = torch.tensor([0])

            data = Data(x = x, edge_index = edge_index, edge_attr = edge_attr, y = y, pos = None)
            return data

new_custom_dataset = NEWCustomDataset()
new_custom_loader = DataLoader(new_custom_dataset, batch_size = 1, shuffle = False)
print(new_custom_dataset)
print('Number of Total Crystal Dataset : {}'.format(len(new_custom_dataset)))
new_sample = next(iter(new_custom_loader))

In [ ]:
# 필요하다면 여기서 모델 Reload 가능!

average_new_test_loss = 0
normed_perovskite_preds = []
denorm_perovskite_preds = []

for batch_data in new_custom_loader:
    batch_data = batch_data.to(device)
    x, edge_index, edge_attr, batch = batch_data.x, batch_data.edge_index, batch_data.edge_attr, batch_data.batch
    target = batch_data.y                           # Pure Target

    pred, h = model(x, edge_index, edge_attr, batch)  # Prediction in normed scale
    normed_perovskite_preds.append(pred)

    assert pred.view(-1).shape == target.shape, 'Shapes are different!!'
    denormed_pred = normalizer.denorm(pred)      # denormed prediction
    denorm_perovskite_preds.append(denormed_pred)

    #new_test_mae = mae(denormed_pred.view(-1), target) # target 값은 0으로 맞춰두어서 의미없고, 실제로는 DFT 계산결과와 비교한다.
    #new_test_loss += new_test_mae.item()  

# average_new_test_loss = new_test_loss / len(new_custom_loader.dataset)
# print('Average Test MAE : {:.6f}'.format(average_new_test_loss))

print(normed_perovskite_preds)
print(denorm_perovskite_preds)
normed_perov_preds = torch.Tensor(normed_perovskite_preds)
denorm_perov_preds = torch.Tensor(denorm_perovskite_preds)

plt.title(mp_id + ' ' + halide_perovskites_mp_ids[mp_id])
plt.plot(list(range(len(denorm_perov_preds))), denorm_perov_preds.cpu().numpy(), label = 'Predictions')
plt.scatter(list(range(len(denorm_perov_preds))), denorm_perov_preds.cpu().numpy())
plt.grid(True)
plt.legend()
plt.show()

# ---------------------- 여기만 DFT 리스트 바꾸면 됨!! -------------------------
plt.title(mp_id + ' ' + halide_perovskites_mp_ids[mp_id])
plt.plot(list(range(len(DFT_CsPbBr3))), DFT_CsPbBr3, label = 'DFT') # DFT 결과 음수이므로 절대값!!
plt.scatter(list(range(len(DFT_CsPbBr3))), DFT_CsPbBr3)
plt.grid(True)
plt.legend()
plt.show()
# ----------------------------------------------------------------------------

#### mp-23037 / CsPbCl3



In [ ]:
# ---------- # 여기만 바꿔주면 된다!! ----------
mp_id = 'mp-23037'   
mp_name = 'CsPbCl3'
changed_atom = 'Pb'
# --------------------------------------------
energy_per_atom_list = []
Test_Material_CIF_list = []
Test_Material_Data = {}
delta = 0.01
num_samples = 4

for i in range(num_samples):

    new_data = {}
    my_API_key = 'I5DdZsmHO3er6WKz'  # You can get your own API key from the homepage of material project.

    with MPRester(my_API_key) as m:
        new_data[mp_id] = (m.query(criteria = {'material_id' : mp_id},
                                   properties = ['energy_per_atom', 'structure']))
    new_structure = new_data[mp_id][0]['structure']
    new_structure[1] = new_structure[1].specie.name, new_structure.frac_coords[1] + (delta * i)

    # 변화된 Structure --> CIF 파일로 저장
    new_structure.to(filename= 'new_{}_{}_delta_{}.cif'.format(mp_name, changed_atom, i))
    Test_Material_CIF_list.append(['new_{}_{}_delta_{}'.format(mp_name, changed_atom, i), 0]) # 물성은 추론할 것이므로 형식상 0
    Test_Material_Data['new_{}_{}_delta_{}'.format(mp_name, changed_atom, i)] = [{'structure' : new_structure,
                                                                                  'energy_per_atom' : 0}]
print('You Must Check the Contents in the "Test_Material_Data"')
print(Test_Material_Data)

In [ ]:
print('Test_Material_CIF_list Check!!')
print(Test_Material_CIF_list)
print()

idx = 2
cif_id, target = Test_Material_CIF_list[idx]
print('CIF ID : {}, Target : {}'.format(cif_id, target))

new_crystal = Test_Material_Data[cif_id][0]['structure'] # 생성된 구조정보 접근
new_crystal

In [ ]:
class NEWCustomDataset(Dataset):
    def __init__(self, new_id_prop_data = Test_Material_CIF_list,
                 embedding_dict = embedding_dict, cutoff_radius = 6, M = 8):
        super().__init__()
        self.new_id_prop_data = new_id_prop_data
        self.embedding_dict   = embedding_dict
        self.cutoff_radius    = cutoff_radius
        self.M = M

    def __len__(self):
        return len(self.new_id_prop_data)

    def __getitem__(self, index):

        cif_id, target = self.new_id_prop_data[index] # idx에 의해 Structure / Target Property 추출
        if cif_id == []:
            return 0
        else:
            crystal = Test_Material_Data[cif_id][0]['structure']
            #crystal = sample[0] # Sample 데이터의 Structure 정보 접근
            target  = sample[1]  # Sample 데이터의 Target Property 정보 접근

            atomic_attributes = [self.embedding_dict[crystal[i].specie.number] for i in range(len(crystal))] # Crystal의 i번째 element number로 접근하여 임베딩벡터 맵핑
            atomic_attributes = np.array(atomic_attributes)

            # 유닛셀 안의 각 원자에 대해 반경 안에 존재하는 이웃노드들의 위치정보
            all_neighbors = crystal.get_all_neighbors(r = self.cutoff_radius, include_index = True)
            # x[1] : 유닛셀 안의 중심노드와 이웃노드 사이의 거리를 기반으로 정렬
            all_neighbors_sorted = [sorted(neighbor, key = lambda x : x[1]) for neighbor in all_neighbors]


            # 이웃노드들의 인덱스 행렬과 특성정보
            neighbor_feature_index = []
            neighbor_features = []
            neighbor_coordinates = []
            max_number_of_neighbors = self.M  # M

            for neighbor in all_neighbors_sorted:
                neighbor_feature_index.append(list(map(lambda x : x[2], neighbor[:max_number_of_neighbors])))             # x[2] : 이웃원자 인덱스
                neighbor_features.append(list(map(lambda x : x[1], neighbor[:max_number_of_neighbors])))                  # x[1] : 중심원자와 이웃원자 사이 거리
                neighbor_coordinates.append(np.array(list(map(lambda x : x.coords, neighbor[:max_number_of_neighbors])))) # 이웃원자 좌표

            # Convert to Numpy Arrays
            neighbor_feature_index = np.array(neighbor_feature_index)
            neighbor_features = np.array(neighbor_features)
            neighbor_coordinates = np.array(neighbor_coordinates)

            # Distance(scalar, Edge Weight) --> Gaussian Function array
            # Gaussian Basis로 확장된 이 거리행렬이 실제 결정구조 내의 모든 Edge에 대한 Edge Attributes로 작용한다.
            neighbor_features_expanded = Gaussian.expand(neighbor_features) # [N, M] --> [N, M, num_filter]

            # --------------------------- Graph Informations ------------------------------
            # Node Informations
            # node_labels = node_labels
            node_attributes = torch.Tensor(atomic_attributes)

            # Edge Informations
            edge_index      = torch.Tensor(neighbor_feature_index)
            edge_attributes = torch.Tensor(neighbor_features_expanded)
            edge_distance   = torch.Tensor(neighbor_features)

            # Target Property and CIF ID
            target_property = torch.Tensor([0])

            # PyG 그래프 데이터로 변환
            # --------------------------------------------------
            N = edge_index.shape[0]
            M = edge_index.shape[1]
            new_edge_index = []
            for i in range(N): # 각각의 행에 대해
                central_node_index = i # 중심노드 인덱스 i에 대해
                for j in range(M):
                    neighbor_node_index = edge_index[i][j]
                    index_tuple = (central_node_index, neighbor_node_index.item()) # Neighbor node index는 tensor이므로 .item()으로 값 추출
                    new_edge_index.append(index_tuple)
            new_edge_index = torch.Tensor(new_edge_index)
            
            # ------------------ undirected edge index ------------------
            undirected_edge_index = []
            for i in range(len(new_edge_index)):
                source_idx, target_idx = new_edge_index[i]
                inverse_edge = torch.Tensor([target_idx.item(), source_idx.item()])
                out = torch.cat([new_edge_index[i], inverse_edge]).view(-1, 2).long().numpy()
                undirected_edge_index.append(out)

            undirected_edge_index = np.array(undirected_edge_index)
            undirected_edge_index = torch.Tensor(undirected_edge_index).view(-1, 2)
            undirected_edge_index = undirected_edge_index.t()

            # ------------------ undirected edge attr ------------------
            new_edge_attributes = edge_attributes.view(-1, edge_attr_dim)
            undirected_edge_attributes = []
            for i in range(len(new_edge_attributes)):
                temp = new_edge_attributes[i].clone()
                out = torch.cat([new_edge_attributes[i], temp]).view(-1, edge_attr_dim).numpy()
                undirected_edge_attributes.append(out)
                
            undirected_edge_attributes = np.array(undirected_edge_attributes)
            undirected_edge_attributes = torch.Tensor(undirected_edge_attributes).view(-1, edge_attr_dim)

            # 4가지 그래프 속성 준비
            x = node_attributes.clone()
            edge_index = undirected_edge_index.long().clone()
            edge_attr = undirected_edge_attributes.clone().float()
            y = torch.tensor([0])

            data = Data(x = x, edge_index = edge_index, edge_attr = edge_attr, y = y, pos = None)
            return data

new_custom_dataset = NEWCustomDataset()
new_custom_loader = DataLoader(new_custom_dataset, batch_size = 1, shuffle = False)
print(new_custom_dataset)
print('Number of Total Crystal Dataset : {}'.format(len(new_custom_dataset)))
new_sample = next(iter(new_custom_loader))

In [ ]:
# 필요하다면 여기서 모델 Reload 가능!

average_new_test_loss = 0
normed_perovskite_preds = []
denorm_perovskite_preds = []

for batch_data in new_custom_loader:
    batch_data = batch_data.to(device)
    x, edge_index, edge_attr, batch = batch_data.x, batch_data.edge_index, batch_data.edge_attr, batch_data.batch
    target = batch_data.y                           # Pure Target

    pred, h = model(x, edge_index, edge_attr, batch)  # Prediction in normed scale
    normed_perovskite_preds.append(pred)

    assert pred.view(-1).shape == target.shape, 'Shapes are different!!'
    denormed_pred = normalizer.denorm(pred)      # denormed prediction
    denorm_perovskite_preds.append(denormed_pred)

    #new_test_mae = mae(denormed_pred.view(-1), target) # target 값은 0으로 맞춰두어서 의미없고, 실제로는 DFT 계산결과와 비교한다.
    #new_test_loss += new_test_mae.item()  

# average_new_test_loss = new_test_loss / len(new_custom_loader.dataset)
# print('Average Test MAE : {:.6f}'.format(average_new_test_loss))

print(normed_perovskite_preds)
print(denorm_perovskite_preds)
normed_perov_preds = torch.Tensor(normed_perovskite_preds)
denorm_perov_preds = torch.Tensor(denorm_perovskite_preds)

plt.title(mp_id + ' ' + halide_perovskites_mp_ids[mp_id])
plt.plot(list(range(len(denorm_perov_preds))), denorm_perov_preds.cpu().numpy(), label = 'Predictions')
plt.scatter(list(range(len(denorm_perov_preds))), denorm_perov_preds.cpu().numpy())
plt.grid(True)
plt.legend()
plt.show()

# ---------------------- 여기만 DFT 리스트 바꾸면 됨!! -------------------------
plt.title(mp_id + ' ' + halide_perovskites_mp_ids[mp_id])
plt.plot(list(range(len(DFT_CsPbCl3))), DFT_CsPbCl3, label = 'DFT') # DFT 결과 음수이므로 절대값!!
plt.scatter(list(range(len(DFT_CsPbCl3))), DFT_CsPbCl3)
plt.grid(True)
plt.legend()
plt.show()
# ----------------------------------------------------------------------------

#### mp-614013 / CsSnI3


In [ ]:
# ---------- # 여기만 바꿔주면 된다!! ----------
mp_id = 'mp-614013'   
mp_name = 'CsSnI3'
changed_atom = 'Sn'
# --------------------------------------------
energy_per_atom_list = []
Test_Material_CIF_list = []
Test_Material_Data = {}
delta = 0.01
num_samples = 4

for i in range(num_samples):

    new_data = {}
    my_API_key = 'I5DdZsmHO3er6WKz'  # You can get your own API key from the homepage of material project.

    with MPRester(my_API_key) as m:
        new_data[mp_id] = (m.query(criteria = {'material_id' : mp_id},
                                   properties = ['energy_per_atom', 'structure']))
    new_structure = new_data[mp_id][0]['structure']
    new_structure[1] = new_structure[1].specie.name, new_structure.frac_coords[1] + (delta * i)

    # 변화된 Structure --> CIF 파일로 저장
    new_structure.to(filename= 'new_{}_{}_delta_{}.cif'.format(mp_name, changed_atom, i))
    Test_Material_CIF_list.append(['new_{}_{}_delta_{}'.format(mp_name, changed_atom, i), 0]) # 물성은 추론할 것이므로 형식상 0
    Test_Material_Data['new_{}_{}_delta_{}'.format(mp_name, changed_atom, i)] = [{'structure' : new_structure,
                                                                                  'energy_per_atom' : 0}]
print('You Must Check the Contents in the "Test_Material_Data"')
print(Test_Material_Data)

In [ ]:
print('Test_Material_CIF_list Check!!')
print(Test_Material_CIF_list)
print()

idx = 2
cif_id, target = Test_Material_CIF_list[idx]
print('CIF ID : {}, Target : {}'.format(cif_id, target))

new_crystal = Test_Material_Data[cif_id][0]['structure'] # 생성된 구조정보 접근
new_crystal

In [ ]:
class NEWCustomDataset(Dataset):
    def __init__(self, new_id_prop_data = Test_Material_CIF_list,
                 embedding_dict = embedding_dict, cutoff_radius = 6, M = 8):
        super().__init__()
        self.new_id_prop_data = new_id_prop_data
        self.embedding_dict   = embedding_dict
        self.cutoff_radius    = cutoff_radius
        self.M = M

    def __len__(self):
        return len(self.new_id_prop_data)

    def __getitem__(self, index):

        cif_id, target = self.new_id_prop_data[index] # idx에 의해 Structure / Target Property 추출
        if cif_id == []:
            return 0
        else:
            crystal = Test_Material_Data[cif_id][0]['structure']
            #crystal = sample[0] # Sample 데이터의 Structure 정보 접근
            target  = sample[1]  # Sample 데이터의 Target Property 정보 접근

            atomic_attributes = [self.embedding_dict[crystal[i].specie.number] for i in range(len(crystal))] # Crystal의 i번째 element number로 접근하여 임베딩벡터 맵핑
            atomic_attributes = np.array(atomic_attributes)

            # 유닛셀 안의 각 원자에 대해 반경 안에 존재하는 이웃노드들의 위치정보
            all_neighbors = crystal.get_all_neighbors(r = self.cutoff_radius, include_index = True)
            # x[1] : 유닛셀 안의 중심노드와 이웃노드 사이의 거리를 기반으로 정렬
            all_neighbors_sorted = [sorted(neighbor, key = lambda x : x[1]) for neighbor in all_neighbors]


            # 이웃노드들의 인덱스 행렬과 특성정보
            neighbor_feature_index = []
            neighbor_features = []
            neighbor_coordinates = []
            max_number_of_neighbors = self.M  # M

            for neighbor in all_neighbors_sorted:
                neighbor_feature_index.append(list(map(lambda x : x[2], neighbor[:max_number_of_neighbors])))             # x[2] : 이웃원자 인덱스
                neighbor_features.append(list(map(lambda x : x[1], neighbor[:max_number_of_neighbors])))                  # x[1] : 중심원자와 이웃원자 사이 거리
                neighbor_coordinates.append(np.array(list(map(lambda x : x.coords, neighbor[:max_number_of_neighbors])))) # 이웃원자 좌표

            # Convert to Numpy Arrays
            neighbor_feature_index = np.array(neighbor_feature_index)
            neighbor_features = np.array(neighbor_features)
            neighbor_coordinates = np.array(neighbor_coordinates)

            # Distance(scalar, Edge Weight) --> Gaussian Function array
            # Gaussian Basis로 확장된 이 거리행렬이 실제 결정구조 내의 모든 Edge에 대한 Edge Attributes로 작용한다.
            neighbor_features_expanded = Gaussian.expand(neighbor_features) # [N, M] --> [N, M, num_filter]

            # --------------------------- Graph Informations ------------------------------
            # Node Informations
            # node_labels = node_labels
            node_attributes = torch.Tensor(atomic_attributes)

            # Edge Informations
            edge_index      = torch.Tensor(neighbor_feature_index)
            edge_attributes = torch.Tensor(neighbor_features_expanded)
            edge_distance   = torch.Tensor(neighbor_features)

            # Target Property and CIF ID
            target_property = torch.Tensor([0])

            # PyG 그래프 데이터로 변환
            # --------------------------------------------------
            N = edge_index.shape[0]
            M = edge_index.shape[1]
            new_edge_index = []
            for i in range(N): # 각각의 행에 대해
                central_node_index = i # 중심노드 인덱스 i에 대해
                for j in range(M):
                    neighbor_node_index = edge_index[i][j]
                    index_tuple = (central_node_index, neighbor_node_index.item()) # Neighbor node index는 tensor이므로 .item()으로 값 추출
                    new_edge_index.append(index_tuple)
            new_edge_index = torch.Tensor(new_edge_index)
            
            # ------------------ undirected edge index ------------------
            undirected_edge_index = []
            for i in range(len(new_edge_index)):
                source_idx, target_idx = new_edge_index[i]
                inverse_edge = torch.Tensor([target_idx.item(), source_idx.item()])
                out = torch.cat([new_edge_index[i], inverse_edge]).view(-1, 2).long().numpy()
                undirected_edge_index.append(out)

            undirected_edge_index = np.array(undirected_edge_index)
            undirected_edge_index = torch.Tensor(undirected_edge_index).view(-1, 2)
            undirected_edge_index = undirected_edge_index.t()

            # ------------------ undirected edge attr ------------------
            new_edge_attributes = edge_attributes.view(-1, edge_attr_dim)
            undirected_edge_attributes = []
            for i in range(len(new_edge_attributes)):
                temp = new_edge_attributes[i].clone()
                out = torch.cat([new_edge_attributes[i], temp]).view(-1, edge_attr_dim).numpy()
                undirected_edge_attributes.append(out)
                
            undirected_edge_attributes = np.array(undirected_edge_attributes)
            undirected_edge_attributes = torch.Tensor(undirected_edge_attributes).view(-1, edge_attr_dim)

            # 4가지 그래프 속성 준비
            x = node_attributes.clone()
            edge_index = undirected_edge_index.long().clone()
            edge_attr = undirected_edge_attributes.clone().float()
            y = torch.tensor([0])

            data = Data(x = x, edge_index = edge_index, edge_attr = edge_attr, y = y, pos = None)
            return data

new_custom_dataset = NEWCustomDataset()
new_custom_loader = DataLoader(new_custom_dataset, batch_size = 1, shuffle = False)
print(new_custom_dataset)
print('Number of Total Crystal Dataset : {}'.format(len(new_custom_dataset)))
new_sample = next(iter(new_custom_loader))

In [ ]:
# 필요하다면 여기서 모델 Reload 가능!

average_new_test_loss = 0
normed_perovskite_preds = []
denorm_perovskite_preds = []

for batch_data in new_custom_loader:
    batch_data = batch_data.to(device)
    x, edge_index, edge_attr, batch = batch_data.x, batch_data.edge_index, batch_data.edge_attr, batch_data.batch
    target = batch_data.y                           # Pure Target

    pred, h = model(x, edge_index, edge_attr, batch)  # Prediction in normed scale
    normed_perovskite_preds.append(pred)

    assert pred.view(-1).shape == target.shape, 'Shapes are different!!'
    denormed_pred = normalizer.denorm(pred)      # denormed prediction
    denorm_perovskite_preds.append(denormed_pred)

    #new_test_mae = mae(denormed_pred.view(-1), target) # target 값은 0으로 맞춰두어서 의미없고, 실제로는 DFT 계산결과와 비교한다.
    #new_test_loss += new_test_mae.item()  

# average_new_test_loss = new_test_loss / len(new_custom_loader.dataset)
# print('Average Test MAE : {:.6f}'.format(average_new_test_loss))

print(normed_perovskite_preds)
print(denorm_perovskite_preds)
normed_perov_preds = torch.Tensor(normed_perovskite_preds)
denorm_perov_preds = torch.Tensor(denorm_perovskite_preds)

plt.title(mp_id + ' ' + halide_perovskites_mp_ids[mp_id])
plt.plot(list(range(len(denorm_perov_preds))), denorm_perov_preds.cpu().numpy(), label = 'Predictions')
plt.scatter(list(range(len(denorm_perov_preds))), denorm_perov_preds.cpu().numpy())
plt.grid(True)
plt.legend()
plt.show()

# ---------------------- 여기만 DFT 리스트 바꾸면 됨!! -------------------------
plt.title(mp_id + ' ' + halide_perovskites_mp_ids[mp_id])
plt.plot(list(range(len(DFT_CsSnI3))), DFT_CsSnI3, label = 'DFT') # DFT 결과 음수이므로 절대값!!
plt.scatter(list(range(len(DFT_CsSnI3))), DFT_CsSnI3)
plt.grid(True)
plt.legend()
plt.show()
# ----------------------------------------------------------------------------

#### mp-1070375 / CsSnCl3



In [ ]:
# ---------- # 여기만 바꿔주면 된다!! ----------
mp_id = 'mp-1070375'   
mp_name = 'CsSnCl3'
changed_atom = 'Sn'
# --------------------------------------------
energy_per_atom_list = []
Test_Material_CIF_list = []
Test_Material_Data = {}
delta = 0.01
num_samples = 4

for i in range(num_samples):

    new_data = {}
    my_API_key = 'I5DdZsmHO3er6WKz'  # You can get your own API key from the homepage of material project.

    with MPRester(my_API_key) as m:
        new_data[mp_id] = (m.query(criteria = {'material_id' : mp_id},
                                   properties = ['energy_per_atom', 'structure']))
    new_structure = new_data[mp_id][0]['structure']
    new_structure[1] = new_structure[1].specie.name, new_structure.frac_coords[1] + (delta * i)

    # 변화된 Structure --> CIF 파일로 저장
    new_structure.to(filename= 'new_{}_{}_delta_{}.cif'.format(mp_name, changed_atom, i))
    Test_Material_CIF_list.append(['new_{}_{}_delta_{}'.format(mp_name, changed_atom, i), 0]) # 물성은 추론할 것이므로 형식상 0
    Test_Material_Data['new_{}_{}_delta_{}'.format(mp_name, changed_atom, i)] = [{'structure' : new_structure,
                                                                                  'energy_per_atom' : 0}]
print('You Must Check the Contents in the "Test_Material_Data"')
print(Test_Material_Data)

In [ ]:
print('Test_Material_CIF_list Check!!')
print(Test_Material_CIF_list)
print()

idx = 2
cif_id, target = Test_Material_CIF_list[idx]
print('CIF ID : {}, Target : {}'.format(cif_id, target))

new_crystal = Test_Material_Data[cif_id][0]['structure'] # 생성된 구조정보 접근
new_crystal

In [ ]:
class NEWCustomDataset(Dataset):
    def __init__(self, new_id_prop_data = Test_Material_CIF_list,
                 embedding_dict = embedding_dict, cutoff_radius = 6, M = 8):
        super().__init__()
        self.new_id_prop_data = new_id_prop_data
        self.embedding_dict   = embedding_dict
        self.cutoff_radius    = cutoff_radius
        self.M = M

    def __len__(self):
        return len(self.new_id_prop_data)

    def __getitem__(self, index):

        cif_id, target = self.new_id_prop_data[index] # idx에 의해 Structure / Target Property 추출
        if cif_id == []:
            return 0
        else:
            crystal = Test_Material_Data[cif_id][0]['structure']
            #crystal = sample[0] # Sample 데이터의 Structure 정보 접근
            target  = sample[1]  # Sample 데이터의 Target Property 정보 접근

            atomic_attributes = [self.embedding_dict[crystal[i].specie.number] for i in range(len(crystal))] # Crystal의 i번째 element number로 접근하여 임베딩벡터 맵핑
            atomic_attributes = np.array(atomic_attributes)

            # 유닛셀 안의 각 원자에 대해 반경 안에 존재하는 이웃노드들의 위치정보
            all_neighbors = crystal.get_all_neighbors(r = self.cutoff_radius, include_index = True)
            # x[1] : 유닛셀 안의 중심노드와 이웃노드 사이의 거리를 기반으로 정렬
            all_neighbors_sorted = [sorted(neighbor, key = lambda x : x[1]) for neighbor in all_neighbors]


            # 이웃노드들의 인덱스 행렬과 특성정보
            neighbor_feature_index = []
            neighbor_features = []
            neighbor_coordinates = []
            max_number_of_neighbors = self.M  # M

            for neighbor in all_neighbors_sorted:
                neighbor_feature_index.append(list(map(lambda x : x[2], neighbor[:max_number_of_neighbors])))             # x[2] : 이웃원자 인덱스
                neighbor_features.append(list(map(lambda x : x[1], neighbor[:max_number_of_neighbors])))                  # x[1] : 중심원자와 이웃원자 사이 거리
                neighbor_coordinates.append(np.array(list(map(lambda x : x.coords, neighbor[:max_number_of_neighbors])))) # 이웃원자 좌표

            # Convert to Numpy Arrays
            neighbor_feature_index = np.array(neighbor_feature_index)
            neighbor_features = np.array(neighbor_features)
            neighbor_coordinates = np.array(neighbor_coordinates)

            # Distance(scalar, Edge Weight) --> Gaussian Function array
            # Gaussian Basis로 확장된 이 거리행렬이 실제 결정구조 내의 모든 Edge에 대한 Edge Attributes로 작용한다.
            neighbor_features_expanded = Gaussian.expand(neighbor_features) # [N, M] --> [N, M, num_filter]

            # --------------------------- Graph Informations ------------------------------
            # Node Informations
            # node_labels = node_labels
            node_attributes = torch.Tensor(atomic_attributes)

            # Edge Informations
            edge_index      = torch.Tensor(neighbor_feature_index)
            edge_attributes = torch.Tensor(neighbor_features_expanded)
            edge_distance   = torch.Tensor(neighbor_features)

            # Target Property and CIF ID
            target_property = torch.Tensor([0])

            # PyG 그래프 데이터로 변환
            # --------------------------------------------------
            N = edge_index.shape[0]
            M = edge_index.shape[1]
            new_edge_index = []
            for i in range(N): # 각각의 행에 대해
                central_node_index = i # 중심노드 인덱스 i에 대해
                for j in range(M):
                    neighbor_node_index = edge_index[i][j]
                    index_tuple = (central_node_index, neighbor_node_index.item()) # Neighbor node index는 tensor이므로 .item()으로 값 추출
                    new_edge_index.append(index_tuple)
            new_edge_index = torch.Tensor(new_edge_index)
            
            # ------------------ undirected edge index ------------------
            undirected_edge_index = []
            for i in range(len(new_edge_index)):
                source_idx, target_idx = new_edge_index[i]
                inverse_edge = torch.Tensor([target_idx.item(), source_idx.item()])
                out = torch.cat([new_edge_index[i], inverse_edge]).view(-1, 2).long().numpy()
                undirected_edge_index.append(out)

            undirected_edge_index = np.array(undirected_edge_index)
            undirected_edge_index = torch.Tensor(undirected_edge_index).view(-1, 2)
            undirected_edge_index = undirected_edge_index.t()

            # ------------------ undirected edge attr ------------------
            new_edge_attributes = edge_attributes.view(-1, edge_attr_dim)
            undirected_edge_attributes = []
            for i in range(len(new_edge_attributes)):
                temp = new_edge_attributes[i].clone()
                out = torch.cat([new_edge_attributes[i], temp]).view(-1, edge_attr_dim).numpy()
                undirected_edge_attributes.append(out)
                
            undirected_edge_attributes = np.array(undirected_edge_attributes)
            undirected_edge_attributes = torch.Tensor(undirected_edge_attributes).view(-1, edge_attr_dim)

            # 4가지 그래프 속성 준비
            x = node_attributes.clone()
            edge_index = undirected_edge_index.long().clone()
            edge_attr = undirected_edge_attributes.clone().float()
            y = torch.tensor([0])

            data = Data(x = x, edge_index = edge_index, edge_attr = edge_attr, y = y, pos = None)
            return data

new_custom_dataset = NEWCustomDataset()
new_custom_loader = DataLoader(new_custom_dataset, batch_size = 1, shuffle = False)
print(new_custom_dataset)
print('Number of Total Crystal Dataset : {}'.format(len(new_custom_dataset)))
new_sample = next(iter(new_custom_loader))

In [ ]:
# 필요하다면 여기서 모델 Reload 가능!

average_new_test_loss = 0
normed_perovskite_preds = []
denorm_perovskite_preds = []

for batch_data in new_custom_loader:
    batch_data = batch_data.to(device)
    x, edge_index, edge_attr, batch = batch_data.x, batch_data.edge_index, batch_data.edge_attr, batch_data.batch
    target = batch_data.y                           # Pure Target

    pred, h = model(x, edge_index, edge_attr, batch)  # Prediction in normed scale
    normed_perovskite_preds.append(pred)

    assert pred.view(-1).shape == target.shape, 'Shapes are different!!'
    denormed_pred = normalizer.denorm(pred)      # denormed prediction
    denorm_perovskite_preds.append(denormed_pred)

    #new_test_mae = mae(denormed_pred.view(-1), target) # target 값은 0으로 맞춰두어서 의미없고, 실제로는 DFT 계산결과와 비교한다.
    #new_test_loss += new_test_mae.item()  

# average_new_test_loss = new_test_loss / len(new_custom_loader.dataset)
# print('Average Test MAE : {:.6f}'.format(average_new_test_loss))

print(normed_perovskite_preds)
print(denorm_perovskite_preds)
normed_perov_preds = torch.Tensor(normed_perovskite_preds)
denorm_perov_preds = torch.Tensor(denorm_perovskite_preds)

plt.title(mp_id + ' ' + halide_perovskites_mp_ids[mp_id])
plt.plot(list(range(len(denorm_perov_preds))), denorm_perov_preds.cpu().numpy(), label = 'Predictions')
plt.scatter(list(range(len(denorm_perov_preds))), denorm_perov_preds.cpu().numpy())
plt.grid(True)
plt.legend()
plt.show()

# ---------------------- 여기만 DFT 리스트 바꾸면 됨!! -------------------------
plt.title(mp_id + ' ' + halide_perovskites_mp_ids[mp_id])
plt.plot(list(range(len(DFT_CsSnCl3))), DFT_CsSnCl3, label = 'DFT') # DFT 결과 음수이므로 절대값!!
plt.scatter(list(range(len(DFT_CsSnCl3))), DFT_CsSnCl3)
plt.grid(True)
plt.legend()
plt.show()
# ----------------------------------------------------------------------------

#### mp-27214 / CsSnBr3



In [ ]:
# ---------- # 여기만 바꿔주면 된다!! ----------
mp_id = 'mp-27214'   
mp_name = 'CsSnBr3'
changed_atom = 'Sn'
# --------------------------------------------
energy_per_atom_list = []
Test_Material_CIF_list = []
Test_Material_Data = {}
delta = 0.01
num_samples = 4

for i in range(num_samples):

    new_data = {}
    my_API_key = 'I5DdZsmHO3er6WKz'  # You can get your own API key from the homepage of material project.

    with MPRester(my_API_key) as m:
        new_data[mp_id] = (m.query(criteria = {'material_id' : mp_id},
                                   properties = ['energy_per_atom', 'structure']))
    new_structure = new_data[mp_id][0]['structure']
    new_structure[1] = new_structure[1].specie.name, new_structure.frac_coords[1] + (delta * i)

    # 변화된 Structure --> CIF 파일로 저장
    new_structure.to(filename= 'new_{}_{}_delta_{}.cif'.format(mp_name, changed_atom, i))
    Test_Material_CIF_list.append(['new_{}_{}_delta_{}'.format(mp_name, changed_atom, i), 0]) # 물성은 추론할 것이므로 형식상 0
    Test_Material_Data['new_{}_{}_delta_{}'.format(mp_name, changed_atom, i)] = [{'structure' : new_structure,
                                                                                  'energy_per_atom' : 0}]
print('You Must Check the Contents in the "Test_Material_Data"')
print(Test_Material_Data)

In [ ]:
print('Test_Material_CIF_list Check!!')
print(Test_Material_CIF_list)
print()

idx = 2
cif_id, target = Test_Material_CIF_list[idx]
print('CIF ID : {}, Target : {}'.format(cif_id, target))

new_crystal = Test_Material_Data[cif_id][0]['structure'] # 생성된 구조정보 접근
new_crystal

In [ ]:
class NEWCustomDataset(Dataset):
    def __init__(self, new_id_prop_data = Test_Material_CIF_list,
                 embedding_dict = embedding_dict, cutoff_radius = 6, M = 8):
        super().__init__()
        self.new_id_prop_data = new_id_prop_data
        self.embedding_dict   = embedding_dict
        self.cutoff_radius    = cutoff_radius
        self.M = M

    def __len__(self):
        return len(self.new_id_prop_data)

    def __getitem__(self, index):

        cif_id, target = self.new_id_prop_data[index] # idx에 의해 Structure / Target Property 추출
        if cif_id == []:
            return 0
        else:
            crystal = Test_Material_Data[cif_id][0]['structure']
            #crystal = sample[0] # Sample 데이터의 Structure 정보 접근
            target  = sample[1]  # Sample 데이터의 Target Property 정보 접근

            atomic_attributes = [self.embedding_dict[crystal[i].specie.number] for i in range(len(crystal))] # Crystal의 i번째 element number로 접근하여 임베딩벡터 맵핑
            atomic_attributes = np.array(atomic_attributes)

            # 유닛셀 안의 각 원자에 대해 반경 안에 존재하는 이웃노드들의 위치정보
            all_neighbors = crystal.get_all_neighbors(r = self.cutoff_radius, include_index = True)
            # x[1] : 유닛셀 안의 중심노드와 이웃노드 사이의 거리를 기반으로 정렬
            all_neighbors_sorted = [sorted(neighbor, key = lambda x : x[1]) for neighbor in all_neighbors]


            # 이웃노드들의 인덱스 행렬과 특성정보
            neighbor_feature_index = []
            neighbor_features = []
            neighbor_coordinates = []
            max_number_of_neighbors = self.M  # M

            for neighbor in all_neighbors_sorted:
                neighbor_feature_index.append(list(map(lambda x : x[2], neighbor[:max_number_of_neighbors])))             # x[2] : 이웃원자 인덱스
                neighbor_features.append(list(map(lambda x : x[1], neighbor[:max_number_of_neighbors])))                  # x[1] : 중심원자와 이웃원자 사이 거리
                neighbor_coordinates.append(np.array(list(map(lambda x : x.coords, neighbor[:max_number_of_neighbors])))) # 이웃원자 좌표

            # Convert to Numpy Arrays
            neighbor_feature_index = np.array(neighbor_feature_index)
            neighbor_features = np.array(neighbor_features)
            neighbor_coordinates = np.array(neighbor_coordinates)

            # Distance(scalar, Edge Weight) --> Gaussian Function array
            # Gaussian Basis로 확장된 이 거리행렬이 실제 결정구조 내의 모든 Edge에 대한 Edge Attributes로 작용한다.
            neighbor_features_expanded = Gaussian.expand(neighbor_features) # [N, M] --> [N, M, num_filter]

            # --------------------------- Graph Informations ------------------------------
            # Node Informations
            # node_labels = node_labels
            node_attributes = torch.Tensor(atomic_attributes)

            # Edge Informations
            edge_index      = torch.Tensor(neighbor_feature_index)
            edge_attributes = torch.Tensor(neighbor_features_expanded)
            edge_distance   = torch.Tensor(neighbor_features)

            # Target Property and CIF ID
            target_property = torch.Tensor([0])

            # PyG 그래프 데이터로 변환
            # --------------------------------------------------
            N = edge_index.shape[0]
            M = edge_index.shape[1]
            new_edge_index = []
            for i in range(N): # 각각의 행에 대해
                central_node_index = i # 중심노드 인덱스 i에 대해
                for j in range(M):
                    neighbor_node_index = edge_index[i][j]
                    index_tuple = (central_node_index, neighbor_node_index.item()) # Neighbor node index는 tensor이므로 .item()으로 값 추출
                    new_edge_index.append(index_tuple)
            new_edge_index = torch.Tensor(new_edge_index)
            
            # ------------------ undirected edge index ------------------
            undirected_edge_index = []
            for i in range(len(new_edge_index)):
                source_idx, target_idx = new_edge_index[i]
                inverse_edge = torch.Tensor([target_idx.item(), source_idx.item()])
                out = torch.cat([new_edge_index[i], inverse_edge]).view(-1, 2).long().numpy()
                undirected_edge_index.append(out)

            undirected_edge_index = np.array(undirected_edge_index)
            undirected_edge_index = torch.Tensor(undirected_edge_index).view(-1, 2)
            undirected_edge_index = undirected_edge_index.t()

            # ------------------ undirected edge attr ------------------
            new_edge_attributes = edge_attributes.view(-1, edge_attr_dim)
            undirected_edge_attributes = []
            for i in range(len(new_edge_attributes)):
                temp = new_edge_attributes[i].clone()
                out = torch.cat([new_edge_attributes[i], temp]).view(-1, edge_attr_dim).numpy()
                undirected_edge_attributes.append(out)
                
            undirected_edge_attributes = np.array(undirected_edge_attributes)
            undirected_edge_attributes = torch.Tensor(undirected_edge_attributes).view(-1, edge_attr_dim)

            # 4가지 그래프 속성 준비
            x = node_attributes.clone()
            edge_index = undirected_edge_index.long().clone()
            edge_attr = undirected_edge_attributes.clone().float()
            y = torch.tensor([0])

            data = Data(x = x, edge_index = edge_index, edge_attr = edge_attr, y = y, pos = None)
            return data

new_custom_dataset = NEWCustomDataset()
new_custom_loader = DataLoader(new_custom_dataset, batch_size = 1, shuffle = False)
print(new_custom_dataset)
print('Number of Total Crystal Dataset : {}'.format(len(new_custom_dataset)))
new_sample = next(iter(new_custom_loader))

In [ ]:
# 필요하다면 여기서 모델 Reload 가능!

average_new_test_loss = 0
normed_perovskite_preds = []
denorm_perovskite_preds = []

for batch_data in new_custom_loader:
    batch_data = batch_data.to(device)
    x, edge_index, edge_attr, batch = batch_data.x, batch_data.edge_index, batch_data.edge_attr, batch_data.batch
    target = batch_data.y                           # Pure Target

    pred, h = model(x, edge_index, edge_attr, batch)  # Prediction in normed scale
    normed_perovskite_preds.append(pred)

    assert pred.view(-1).shape == target.shape, 'Shapes are different!!'
    denormed_pred = normalizer.denorm(pred)      # denormed prediction
    denorm_perovskite_preds.append(denormed_pred)

    #new_test_mae = mae(denormed_pred.view(-1), target) # target 값은 0으로 맞춰두어서 의미없고, 실제로는 DFT 계산결과와 비교한다.
    #new_test_loss += new_test_mae.item()  

# average_new_test_loss = new_test_loss / len(new_custom_loader.dataset)
# print('Average Test MAE : {:.6f}'.format(average_new_test_loss))

print(normed_perovskite_preds)
print(denorm_perovskite_preds)
normed_perov_preds = torch.Tensor(normed_perovskite_preds)
denorm_perov_preds = torch.Tensor(denorm_perovskite_preds)

plt.title(mp_id + ' ' + halide_perovskites_mp_ids[mp_id])
plt.plot(list(range(len(denorm_perov_preds))), denorm_perov_preds.cpu().numpy(), label = 'Predictions')
plt.scatter(list(range(len(denorm_perov_preds))), denorm_perov_preds.cpu().numpy())
plt.grid(True)
plt.legend()
plt.show()

# ---------------------- 여기만 DFT 리스트 바꾸면 됨!! -------------------------
plt.title(mp_id + ' ' + halide_perovskites_mp_ids[mp_id])
plt.plot(list(range(len(DFT_CsSnBr3))), DFT_CsSnBr3, label = 'DFT') # DFT 결과 음수이므로 절대값!!
plt.scatter(list(range(len(DFT_CsSnBr3))), DFT_CsSnBr3)
plt.grid(True)
plt.legend()
plt.show()
# ----------------------------------------------------------------------------

## ========== Scalability of GNN ==========




In [ ]:
%cd /content/drive/MyDrive/cgcnn/magnified_perovskites/
%ls

In [ ]:
import os
import glob
from tqdm import tqdm

path = './*' # 3600개의 계산 디렉터리가 압축해제되어 있는 현재 위치
file_list = glob.glob(path)
dir_list = [element for element in file_list if element.endswith('.vasp')] # 계산 디렉터리 리스트 추출
dir_list_sorted = sorted(dir_list) # 정렬
dir_list_sorted

In [ ]:
idx = 4

from pymatgen.io.vasp import Poscar
poscar = Poscar.from_file(dir_list_sorted[idx])
crystal = poscar.structure

In [ ]:
crystal[1]

In [ ]:
name = dir_list_sorted[idx]


Test_Material_CIF_list = []
Test_Material_Data = {}
delta = 0.01
num_samples = 4

for i in range(num_samples):
    poscar = Poscar.from_file(name)
    crystal = poscar.structure
    crystal[1] = crystal[1].specie.name, crystal.frac_coords[1] + (delta * i)

    # 변화된 Structure --> CIF 파일로 저장
    crystal.to(filename= '{}_delta_{}.cif'.format(name, i))
    Test_Material_CIF_list.append(['{}_delta_{}'.format(name, i), 0]) # 물성은 추론할 것이므로 형식상 0
    Test_Material_Data['{}_delta_{}'.format(name, i)] = [{'structure' : crystal, 'energy_per_atom' : 0}]

print('You Must Check the Contents in the "Test_Material_Data"')
print(Test_Material_Data)


In [ ]:
print('Test_Material_CIF_list Check!!')
print(Test_Material_CIF_list)
print()

idx = 2
cif_id, target = Test_Material_CIF_list[idx]
print('CIF ID : {}, Target : {}'.format(cif_id, target))

new_crystal = Test_Material_Data[cif_id][0]['structure'] # 생성된 구조정보 접근
new_crystal

In [ ]:
class NEWCustomDataset(Dataset):
    def __init__(self, new_id_prop_data = Test_Material_CIF_list,
                 embedding_dict = embedding_dict, cutoff_radius = 6, M = 8):
        super().__init__()
        self.new_id_prop_data = new_id_prop_data
        self.embedding_dict   = embedding_dict
        self.cutoff_radius    = cutoff_radius
        self.M = M

    def __len__(self):
        return len(self.new_id_prop_data)

    def __getitem__(self, index):

        cif_id, target = self.new_id_prop_data[index] # idx에 의해 Structure / Target Property 추출
        if cif_id == []:
            return 0
        else:
            crystal = Test_Material_Data[cif_id][0]['structure']
            #crystal = sample[0] # Sample 데이터의 Structure 정보 접근
            target  = sample[1]  # Sample 데이터의 Target Property 정보 접근

            atomic_attributes = [self.embedding_dict[crystal[i].specie.number] for i in range(len(crystal))] # Crystal의 i번째 element number로 접근하여 임베딩벡터 맵핑
            atomic_attributes = np.array(atomic_attributes)

            # 유닛셀 안의 각 원자에 대해 반경 안에 존재하는 이웃노드들의 위치정보
            all_neighbors = crystal.get_all_neighbors(r = self.cutoff_radius, include_index = True)
            # x[1] : 유닛셀 안의 중심노드와 이웃노드 사이의 거리를 기반으로 정렬
            all_neighbors_sorted = [sorted(neighbor, key = lambda x : x[1]) for neighbor in all_neighbors]


            # 이웃노드들의 인덱스 행렬과 특성정보
            neighbor_feature_index = []
            neighbor_features = []
            neighbor_coordinates = []
            max_number_of_neighbors = self.M  # M

            for neighbor in all_neighbors_sorted:
                neighbor_feature_index.append(list(map(lambda x : x[2], neighbor[:max_number_of_neighbors])))             # x[2] : 이웃원자 인덱스
                neighbor_features.append(list(map(lambda x : x[1], neighbor[:max_number_of_neighbors])))                  # x[1] : 중심원자와 이웃원자 사이 거리
                neighbor_coordinates.append(np.array(list(map(lambda x : x.coords, neighbor[:max_number_of_neighbors])))) # 이웃원자 좌표

            # Convert to Numpy Arrays
            neighbor_feature_index = np.array(neighbor_feature_index)
            neighbor_features = np.array(neighbor_features)
            neighbor_coordinates = np.array(neighbor_coordinates)

            # Distance(scalar, Edge Weight) --> Gaussian Function array
            # Gaussian Basis로 확장된 이 거리행렬이 실제 결정구조 내의 모든 Edge에 대한 Edge Attributes로 작용한다.
            neighbor_features_expanded = Gaussian.expand(neighbor_features) # [N, M] --> [N, M, num_filter]

            # --------------------------- Graph Informations ------------------------------
            # Node Informations
            # node_labels = node_labels
            node_attributes = torch.Tensor(atomic_attributes)

            # Edge Informations
            edge_index      = torch.Tensor(neighbor_feature_index)
            edge_attributes = torch.Tensor(neighbor_features_expanded)
            edge_distance   = torch.Tensor(neighbor_features)

            # Target Property and CIF ID
            target_property = torch.Tensor([0])

            # PyG 그래프 데이터로 변환
            # --------------------------------------------------
            N = edge_index.shape[0]
            M = edge_index.shape[1]
            new_edge_index = []
            for i in range(N): # 각각의 행에 대해
                central_node_index = i # 중심노드 인덱스 i에 대해
                for j in range(M):
                    neighbor_node_index = edge_index[i][j]
                    index_tuple = (central_node_index, neighbor_node_index.item()) # Neighbor node index는 tensor이므로 .item()으로 값 추출
                    new_edge_index.append(index_tuple)
            new_edge_index = torch.Tensor(new_edge_index)
            
            # ------------------ undirected edge index ------------------
            undirected_edge_index = []
            for i in range(len(new_edge_index)):
                source_idx, target_idx = new_edge_index[i]
                inverse_edge = torch.Tensor([target_idx.item(), source_idx.item()])
                out = torch.cat([new_edge_index[i], inverse_edge]).view(-1, 2).long().numpy()
                undirected_edge_index.append(out)

            undirected_edge_index = np.array(undirected_edge_index)
            undirected_edge_index = torch.Tensor(undirected_edge_index).view(-1, 2)
            undirected_edge_index = undirected_edge_index.t()

            # ------------------ undirected edge attr ------------------
            new_edge_attributes = edge_attributes.view(-1, edge_attr_dim)
            undirected_edge_attributes = []
            for i in range(len(new_edge_attributes)):
                temp = new_edge_attributes[i].clone()
                out = torch.cat([new_edge_attributes[i], temp]).view(-1, edge_attr_dim).numpy()
                undirected_edge_attributes.append(out)
                
            undirected_edge_attributes = np.array(undirected_edge_attributes)
            undirected_edge_attributes = torch.Tensor(undirected_edge_attributes).view(-1, edge_attr_dim)

            # 4가지 그래프 속성 준비
            x = node_attributes.clone()
            edge_index = undirected_edge_index.long().clone()
            edge_attr = undirected_edge_attributes.clone().float()
            y = torch.tensor([0])

            data = Data(x = x, edge_index = edge_index, edge_attr = edge_attr, y = y, pos = None)
            return data

new_custom_dataset = NEWCustomDataset()
new_custom_loader = DataLoader(new_custom_dataset, batch_size = 1, shuffle = False)
print(new_custom_dataset)
print('Number of Total Crystal Dataset : {}'.format(len(new_custom_dataset)))
new_sample = next(iter(new_custom_loader))

In [ ]:
# 필요하다면 여기서 모델 Reload 가능!

average_new_test_loss = 0
normed_perovskite_preds = []
denorm_perovskite_preds = []

for batch_data in new_custom_loader:
    batch_data = batch_data.to(device)
    x, edge_index, edge_attr, batch = batch_data.x, batch_data.edge_index, batch_data.edge_attr, batch_data.batch
    target = batch_data.y                           # Pure Target

    pred, h = model(x, edge_index, edge_attr, batch)  # Prediction in normed scale
    normed_perovskite_preds.append(pred)

    assert pred.view(-1).shape == target.shape, 'Shapes are different!!'
    denormed_pred = normalizer.denorm(pred)      # denormed prediction
    denorm_perovskite_preds.append(denormed_pred)

    #new_test_mae = mae(denormed_pred.view(-1), target) # target 값은 0으로 맞춰두어서 의미없고, 실제로는 DFT 계산결과와 비교한다.
    #new_test_loss += new_test_mae.item()  

# average_new_test_loss = new_test_loss / len(new_custom_loader.dataset)
# print('Average Test MAE : {:.6f}'.format(average_new_test_loss))

print(normed_perovskite_preds)
print(denorm_perovskite_preds)
normed_perov_preds = torch.Tensor(normed_perovskite_preds)
denorm_perov_preds = torch.Tensor(denorm_perovskite_preds)

plt.title(name)
plt.plot(list(range(len(denorm_perov_preds))), denorm_perov_preds.cpu().numpy(), label = 'Predictions')
plt.scatter(list(range(len(denorm_perov_preds))), denorm_perov_preds.cpu().numpy())
plt.grid(True)
plt.legend()
plt.show()

# ---------------------- 여기만 DFT 리스트 바꾸면 됨!! -------------------------
plt.title(name)
plt.plot(list(range(len(DFT_CsPbI3))), DFT_CsPbI3, label = 'DFT') # DFT 결과 음수이므로 절대값!!
plt.scatter(list(range(len(DFT_CsPbI3))), DFT_CsPbI3)
plt.grid(True)
plt.legend()
plt.show()
# ----------------------------------------------------------------------------